In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import os
import time

get_url_final function

In [2]:
#this function just gets the html code that we need to scrape for data, updated version that is not specific to salah 
def get_url_final(code, year_range, category, player):
    base_url = 'https://fbref.com/en/players/{}/matchlogs/{}/c9/{}/{}-Match-Logs'
    url = base_url.format(code, year_range, category, player)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    html_filtered = soup.find('tbody')
    return(html_filtered)

get_dat, get_date and get_matchweek functions

In [3]:
def get_dat(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('td', {'data-stat': col}).text)
    return (coldat)   

def get_date(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('th', {'data-stat': col}).text)
    return (coldat)   

def get_matchweek(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('td', {'data-stat': col}).text)
    coldat = [coldat.replace('Matchweek ', '') for coldat in coldat]
    return(coldat)

get_data function

In [5]:
#now, we want to modify the function above, to allow us to get any players data that we want, not just salah

def get_data(code, year_range, player, fpl_name):
    data_summary = get_url_final(code, year_range, '', player)
    
    #this if statement is checking whether or not this player actually has data for the particular year. In the case that they do not actually have
    #data for this year, just return an empty dataframe 
    if not data_summary:
        empty = pd.DataFrame()
        return(empty)
    
    data_passing = get_url_final(code, year_range, 'passing', player)
    data_passingtype = get_url_final(code, year_range, 'passing_types', player)
    data_gca = get_url_final(code, year_range, 'gca', player)
    data_defense = get_url_final(code, year_range, 'defense', player)
    data_possession = get_url_final(code, year_range, 'possession', player)
    
    ################################################################
    #getting the data for the necessary columns
    
    ##### summary data #####
    #misc data 
    date = get_date(data_summary, 'date')
    day = get_dat(data_summary,'dayofweek')
    matchweek = get_matchweek(data_summary, 'round')
    venue = get_dat(data_summary, 'venue')
    result = get_dat(data_summary, 'result')
    team = get_dat(data_summary, 'team')
    opponent = get_dat(data_summary, 'opponent')
    start = get_dat(data_summary, 'game_started')
    position = get_dat(data_summary, 'position')
    mins = get_dat(data_summary, 'minutes')

    #performance data
    goals = get_dat(data_summary, 'goals')
    assist = get_dat(data_summary, 'assists')
    pen_goals = get_dat(data_summary, 'pens_made')
    pen_attempted = get_dat(data_summary, 'pens_att')
    shots = get_dat(data_summary, 'shots')
    sot = get_dat(data_summary, 'shots_on_target')
    yellow = get_dat(data_summary, 'cards_yellow')
    red = get_dat(data_summary, 'cards_red')
    touches = get_dat(data_summary, 'touches')
    tackles = get_dat(data_summary, 'tackles')
    interceptions = get_dat(data_summary, 'interceptions')
    blocks = get_dat(data_summary, 'blocks')

    #expected performance metrics
    xg = get_dat(data_summary, 'xg')
    npxg = get_dat(data_summary, 'npxg')
    xag = get_dat(data_summary, 'xg_assist')

    #shot creating actions
    sca = get_dat(data_summary, 'sca')
    gca = get_dat(data_summary, 'gca')

    #passes
    passes_completed = get_dat(data_summary, 'passes_completed')
    passes_attempted = get_dat(data_summary, 'passes')
    prg_passes = get_dat(data_summary, 'progressive_passes')

    #carries 
    carries = get_dat(data_summary, 'carries')
    prg_carries = get_dat(data_summary, 'progressive_carries')

    #take-ons
    takeon_att = get_dat(data_summary, 'take_ons')
    takeon_suc = get_dat(data_summary, 'take_ons_won')
    
    ##### passing data #####
    #total
    pass_distance = get_dat(data_passing, 'passes_total_distance')
    prg_pass_distance = get_dat(data_passing, 'passes_progressive_distance')

    #short
    short_pass_completed = get_dat(data_passing, 'passes_completed_short')
    short_pass_attempted = get_dat(data_passing, 'passes_short')

    #medium 
    med_pass_completed = get_dat(data_passing, 'passes_completed_medium')
    med_pass_attempted = get_dat(data_passing, 'passes_medium')

    #long
    long_pass_completed = get_dat(data_passing, 'passes_completed_long')
    long_pass_attempted = get_dat(data_passing, 'passes_long')

    #misc
    xa = get_dat(data_passing, 'pass_xa')
    keypass = get_dat(data_passing, 'assisted_shots')
    finalthird_pass = get_dat(data_passing, 'passes_into_final_third')
    penaltyarea_pass = get_dat(data_passing, 'passes_into_penalty_area')
    penaltyarea_cross = get_dat(data_passing, 'crosses_into_penalty_area')
    
    
    ##### passing types data #####
    live_pass = get_dat(data_passingtype, 'passes_live')
    dead_pass = get_dat(data_passingtype, 'passes_dead')
    freekick_pass = get_dat(data_passingtype, 'passes_free_kicks')
    through_balls = get_dat(data_passingtype, 'through_balls')
    switches = get_dat(data_passingtype, 'passes_switches')
    crosses = get_dat(data_passingtype, 'crosses')
    throwin = get_dat(data_passingtype, 'throw_ins')
    corners = get_dat(data_passingtype, 'corner_kicks')
    offside_pass = get_dat(data_passingtype, 'passes_offsides')
    
    
    ##### gca data #####
    #sca types 
    live_sca = get_dat(data_gca, 'sca_passes_live')
    deadball_sca = get_dat(data_gca, 'sca_passes_dead')
    takeons_sca = get_dat(data_gca, 'sca_take_ons')
    shots_sca = get_dat(data_gca, 'sca_shots')
    fouls_sca = get_dat(data_gca, 'sca_fouled')
    defense_sca = get_dat(data_gca, 'sca_defense')

    #gca types 
    live_gca = get_dat(data_gca, 'gca_passes_live')
    deadball_gca = get_dat(data_gca, 'gca_passes_dead')
    takeons_gca = get_dat(data_gca, 'gca_take_ons')
    shots_gca = get_dat(data_gca, 'gca_shots')
    fouls_gca = get_dat(data_gca, 'gca_fouled')
    defense_gca = get_dat(data_gca, 'gca_defense')
    
    
    ##### defensive data ######
    #tackles
    tackles_won = get_dat(data_defense,'tackles_won')
    tackles_def = get_dat(data_defense, 'tackles_def_3rd')
    tackles_mid = get_dat(data_defense, 'tackles_mid_3rd')
    tackles_att = get_dat(data_defense, 'tackles_att_3rd')

    #challenges 
    dribblers_tackled = get_dat(data_defense, 'challenge_tackles')
    dribbles_challenged = get_dat(data_defense, 'challenges')
    challenges_lost = get_dat(data_defense, 'challenges_lost')

    #blocks 
    shots_blocked = get_dat(data_defense, 'blocked_shots')
    pass_blocked = get_dat(data_defense, 'blocked_passes')
    clearances = get_dat(data_defense, 'clearances')
    def_errors = get_dat(data_defense, 'errors')
    
    
    ##### possession data #####
    #touches 
    def_penarea_touches = get_dat(data_possession, 'touches_def_pen_area')
    def_third_touches = get_dat(data_possession, 'touches_def_3rd')
    mid_third_touches = get_dat(data_possession, 'touches_mid_3rd')
    att_third_touches = get_dat(data_possession, 'touches_att_3rd')
    pen_area_touches = get_dat(data_possession, 'touches_att_pen_area')

    #carries
    carry_distance = get_dat(data_possession, 'carries_distance')
    prg_carry_distance = get_dat(data_possession, 'carries_progressive_distance')
    final_third_carries = get_dat(data_possession, 'carries_into_final_third')
    pen_area_carries = get_dat(data_possession, 'carries_into_penalty_area')
    miscontrols = get_dat(data_possession, 'miscontrols')
    dispossesed = get_dat(data_possession, 'dispossessed')

    #receiving 
    pass_received = get_dat(data_possession, 'passes_received')
    prg_pass_received = get_dat(data_possession, 'progressive_passes_received')


    ###########################################################
    #creating dataframe
    df = pd.DataFrame({
        'Date': date,
        'Day': day,
        'Matchweek': matchweek,
        'Venue': venue,
        'Result': result,
        'Team': team,
        'Opponent': opponent, 
        'Start': start, 
        'Position': position,
        'Minutes Played': mins, 
        'Goals': goals, 
        'Assists': assist, 
        'Penalties Scored': pen_goals, 
        'Penalties Attempted': pen_attempted, 
        'Shots': shots, 
        'Shots on Target': sot, 
        'Yellow Cards': yellow, 
        'Red Cards': red, 
        'Touches': touches, 
        'Tackles': tackles,
        'Interceptions': interceptions, 
        'Blocks': blocks, 
        'xG': xg,
        'npxG': npxg,
        'xAG': xag, 
        'Shot Creating Actions': sca, 
        'Goal Creating Actions': gca, 
        'Passes Completed': passes_completed, 
        'Passes Attempted': passes_attempted, 
        'Progressive Passes': prg_passes,
        'Carries': carries, 
        'Progressive Carries': prg_carries, 
        'Take-ons Attempted': takeon_att, 
        'Successful Take-ons': takeon_suc,
        
        'Passing Distance': pass_distance, 
        'Progressive Passing Distance': prg_pass_distance,
        'Short Passes Completed' : short_pass_completed,
        'Short Passes Attempted': short_pass_attempted,
        'Medium Passes Completed': med_pass_completed,
        'Medium Passes Attempted': med_pass_attempted,
        'Long Passes Completed': long_pass_completed,
        'Long Passes Attempted': long_pass_attempted,
        'Expected Assists': xa,
        'Key Passes': keypass,
        'Passes into Final Third': finalthird_pass,
        'Passes into Penalty Area': penaltyarea_pass,
        'Crosses into Penalty Area': penaltyarea_cross,
    
        'Live Pass': live_pass,
        'Dead Pass': dead_pass,
        'Free Kick Pass': freekick_pass,
        'Through Balls': through_balls,
        'Switches': switches,
        'Crosses': crosses,
        'Throw Ins Taken': throwin,
        'Corners Taken': corners,
        'Passes Offside': offside_pass,
    
        'Live SCA': live_sca,
        'Deadball SCA': deadball_sca,
        'Take-on SCA': takeons_sca,
        'Shot SCA': shots_sca,
        'Foul SCA': fouls_sca,
        'Defense SCA': defense_sca,
    
        'Live GCA': live_gca,
        'Deadball GCA': deadball_gca,
        'Take-on GCA': takeons_gca,
        'Shot GCA': shots_gca,
        'Foul GCA': fouls_gca,
        'Defense GCA': defense_gca,
    
        'Tackles Won': tackles_won,
        'Defensive Third Tackles': tackles_def,
        'Middle Third Tackles': tackles_mid,
        'Attacking Third Tackles': tackles_att,
        'Dribblers Tackled': dribblers_tackled,
        'Dribblers Tackled Attempts': dribbles_challenged,
        'Challenges Lost': challenges_lost,
        'Shots Blocked': shots_blocked,
        'Passes Blocked': pass_blocked,
        'Clearances': clearances,
        'Defensive Errors': def_errors,
    
        'Defensive Penalty Area Touches': def_penarea_touches,
        'Defensive Third Touches': def_third_touches,
        'Middle Third Touches': mid_third_touches,
        'Attacking Third Touches': att_third_touches,
        'Penalty Area Touches': pen_area_touches,
        'Carry Distance': carry_distance,
        'Progressive Carry Distance': prg_carry_distance,
        'Final Third Carries': final_third_carries,
        'Carries into Penalty Area': pen_area_carries,
        'Miscontrols': miscontrols,
        'Dispossessed': dispossesed,
        'Passes Received': pass_received,
        'Progressive Passes Received': prg_pass_received
    })
    
    #modifying data types
    df = df.astype({
        'Date': 'datetime64[ns]',
        'Day': 'object',
        'Matchweek': 'int64',
        'Venue': 'object',
        'Result': 'object',
        'Team': 'object',
        'Opponent': 'object', 
        'Start': 'object', 
        'Position': 'object',
        'Minutes Played': 'int64', 
        'Goals': 'int64', 
        'Assists': 'int64', 
        'Penalties Scored': 'int64', 
        'Penalties Attempted': 'int64', 
        'Shots': 'int64', 
        'Shots on Target': 'int64', 
        'Yellow Cards': 'int64', 
        'Red Cards': 'int64', 
        'Touches': 'int64', 
        'Tackles': 'int64',
        'Interceptions': 'int64', 
        'Blocks': 'int64', 
        'xG': 'float64',
        'npxG': 'float64',
        'xAG': 'float64', 
        'Shot Creating Actions': 'int64', 
        'Goal Creating Actions': 'int64', 
        'Passes Completed': 'int64', 
        'Passes Attempted': 'int64', 
        'Progressive Passes': 'int64',
        'Carries': 'int64', 
        'Progressive Carries': 'int64', 
        'Take-ons Attempted': 'int64', 
        'Successful Take-ons': 'int64',
        
        'Short Passes Completed' : 'int64',
        'Short Passes Attempted': 'int64',
        'Medium Passes Completed': 'int64',
        'Medium Passes Attempted': 'int64',
        'Long Passes Completed': 'int64',
        'Long Passes Attempted': 'int64',
        'Expected Assists': 'float64',
        'Key Passes': 'int64',
        'Passes into Final Third': 'int64',
        'Passes into Penalty Area': 'int64',
        'Crosses into Penalty Area': 'int64',
    
        'Live Pass': 'int64',
        'Dead Pass': 'int64',
        'Free Kick Pass': 'int64',
        'Through Balls': 'int64',
        'Switches': 'int64',
        'Crosses': 'int64',
        'Throw Ins Taken': 'int64',
        'Corners Taken': 'int64',
        'Passes Offside': 'int64',
    
        'Live SCA': 'int64',
        'Deadball SCA': 'int64',
        'Take-on SCA': 'int64',
        'Shot SCA': 'int64',
        'Foul SCA': 'int64',
        'Defense SCA': 'int64',
    
        'Live GCA': 'int64',
        'Deadball GCA': 'int64',
        'Take-on GCA': 'int64',
        'Shot GCA': 'int64',
        'Foul GCA': 'int64',
        'Defense GCA': 'int64',
    
        'Tackles Won': 'int64',
        'Defensive Third Tackles': 'int64',
        'Middle Third Tackles': 'int64',
        'Attacking Third Tackles': 'int64',
        'Dribblers Tackled': 'int64',
        'Dribblers Tackled Attempts': 'int64',
        'Challenges Lost': 'int64',
        'Shots Blocked': 'int64',
        'Passes Blocked': 'int64',
        'Clearances': 'int64',
        'Defensive Errors': 'int64',
    
        'Defensive Penalty Area Touches': 'int64',
        'Defensive Third Touches': 'int64',
        'Middle Third Touches': 'int64',
        'Attacking Third Touches': 'int64',
        'Penalty Area Touches': 'int64',
        'Carry Distance': 'float64',
        'Progressive Carry Distance': 'float64',
        'Final Third Carries': 'int64',
        'Carries into Penalty Area': 'int64',
        'Miscontrols': 'int64',
        'Dispossessed': 'int64',
        'Passes Received': 'int64',
        'Progressive Passes Received': 'int64'
    })
    
    #changing date column data type 
    df['Date'] = df['Date'].dt.date
    
    ###############################################################################
    
    #now we want to get the fpl data 
    start_year = year_range[:4] #getting the whole of the first year 
    end_year = year_range[-2:]
    format_year = f'{start_year}-{end_year}'
    directory = f'Fantasy-Premier-League/data/{format_year}/players'
    files = os.listdir(directory)
    csv_file = [f for f in files if fpl_name in f]
    fpldf = pd.read_csv(os.path.join(directory, csv_file[0]) + '/gw.csv')

    #some of the columns are duplicated (from the webscraped data), so we want to remove these 
    fpldf = fpldf.drop(['assists', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'fixture',
                                          'goals_conceded', 'goals_scored', 'penalties_missed', 'penalties_saved', 'red_cards'
                                          ,'team_a_score', 'team_h_score', 'was_home', 'yellow_cards', 'element', 'opponent_team', 'starts'
                                          ,'expected_goals_conceded'], axis=1, 
                       errors = 'ignore')

    #we now want to change the 'kickoff time' feature into a datetime object 
    fpldf['kickoff_time'] = pd.to_datetime(fpldf['kickoff_time'])

    #creating new column, which is just the kickoff date 
    fpldf['kickoff_date'] = fpldf['kickoff_time'].dt.date
    
    finaldf = pd.merge(df, fpldf, left_on='Date', right_on='kickoff_date', how='inner')
    
    return(finaldf)

In [6]:
#this is just to use the get_data function to create a representative dataframe, with the purpose being just to get the column names. This is why 
#we have to specify another get_data function below (get_data_final), which implements functionality that deals with players who have empty pages in 
#Fbref
test = get_data('66b76d44', '2022-2023', 'Emi-Buendia', 'Emiliano_Buen')
column_names = test.columns

In [7]:
#this is the final version of get_data, which includes additional functionality to deal with players who hav eempty pages in Fbref
def get_data_final(code, year_range, player, fpl_name):
    data_summary = get_url_final(code, year_range, '', player)
    
    #this if statement is checking whether or not this player actually has data for the particular year. In the case that they do not actually have
    #data for this year, just return an empty dataframe 
    if not data_summary:
        empty = pd.DataFrame(columns = column_names)
        return(empty)
    
    data_passing = get_url_final(code, year_range, 'passing', player)
    data_passingtype = get_url_final(code, year_range, 'passing_types', player)
    data_gca = get_url_final(code, year_range, 'gca', player)
    data_defense = get_url_final(code, year_range, 'defense', player)
    data_possession = get_url_final(code, year_range, 'possession', player)
    
    ################################################################
    #getting the data for the necessary columns
    
    ##### summary data #####
    #misc data 
    date = get_date(data_summary, 'date')
    day = get_dat(data_summary,'dayofweek')
    matchweek = get_matchweek(data_summary, 'round')
    venue = get_dat(data_summary, 'venue')
    result = get_dat(data_summary, 'result')
    team = get_dat(data_summary, 'team')
    opponent = get_dat(data_summary, 'opponent')
    start = get_dat(data_summary, 'game_started')
    position = get_dat(data_summary, 'position')
    mins = get_dat(data_summary, 'minutes')

    #performance data
    goals = get_dat(data_summary, 'goals')
    assist = get_dat(data_summary, 'assists')
    pen_goals = get_dat(data_summary, 'pens_made')
    pen_attempted = get_dat(data_summary, 'pens_att')
    shots = get_dat(data_summary, 'shots')
    sot = get_dat(data_summary, 'shots_on_target')
    yellow = get_dat(data_summary, 'cards_yellow')
    red = get_dat(data_summary, 'cards_red')
    touches = get_dat(data_summary, 'touches')
    tackles = get_dat(data_summary, 'tackles')
    interceptions = get_dat(data_summary, 'interceptions')
    blocks = get_dat(data_summary, 'blocks')

    #expected performance metrics
    xg = get_dat(data_summary, 'xg')
    npxg = get_dat(data_summary, 'npxg')
    xag = get_dat(data_summary, 'xg_assist')

    #shot creating actions
    sca = get_dat(data_summary, 'sca')
    gca = get_dat(data_summary, 'gca')

    #passes
    passes_completed = get_dat(data_summary, 'passes_completed')
    passes_attempted = get_dat(data_summary, 'passes')
    prg_passes = get_dat(data_summary, 'progressive_passes')

    #carries 
    carries = get_dat(data_summary, 'carries')
    prg_carries = get_dat(data_summary, 'progressive_carries')

    #take-ons
    takeon_att = get_dat(data_summary, 'take_ons')
    takeon_suc = get_dat(data_summary, 'take_ons_won')
    
    ##### passing data #####
    #total
    pass_distance = get_dat(data_passing, 'passes_total_distance')
    prg_pass_distance = get_dat(data_passing, 'passes_progressive_distance')

    #short
    short_pass_completed = get_dat(data_passing, 'passes_completed_short')
    short_pass_attempted = get_dat(data_passing, 'passes_short')

    #medium 
    med_pass_completed = get_dat(data_passing, 'passes_completed_medium')
    med_pass_attempted = get_dat(data_passing, 'passes_medium')

    #long
    long_pass_completed = get_dat(data_passing, 'passes_completed_long')
    long_pass_attempted = get_dat(data_passing, 'passes_long')

    #misc
    xa = get_dat(data_passing, 'pass_xa')
    keypass = get_dat(data_passing, 'assisted_shots')
    finalthird_pass = get_dat(data_passing, 'passes_into_final_third')
    penaltyarea_pass = get_dat(data_passing, 'passes_into_penalty_area')
    penaltyarea_cross = get_dat(data_passing, 'crosses_into_penalty_area')
    
    
    ##### passing types data #####
    live_pass = get_dat(data_passingtype, 'passes_live')
    dead_pass = get_dat(data_passingtype, 'passes_dead')
    freekick_pass = get_dat(data_passingtype, 'passes_free_kicks')
    through_balls = get_dat(data_passingtype, 'through_balls')
    switches = get_dat(data_passingtype, 'passes_switches')
    crosses = get_dat(data_passingtype, 'crosses')
    throwin = get_dat(data_passingtype, 'throw_ins')
    corners = get_dat(data_passingtype, 'corner_kicks')
    offside_pass = get_dat(data_passingtype, 'passes_offsides')
    
    
    ##### gca data #####
    #sca types 
    live_sca = get_dat(data_gca, 'sca_passes_live')
    deadball_sca = get_dat(data_gca, 'sca_passes_dead')
    takeons_sca = get_dat(data_gca, 'sca_take_ons')
    shots_sca = get_dat(data_gca, 'sca_shots')
    fouls_sca = get_dat(data_gca, 'sca_fouled')
    defense_sca = get_dat(data_gca, 'sca_defense')

    #gca types 
    live_gca = get_dat(data_gca, 'gca_passes_live')
    deadball_gca = get_dat(data_gca, 'gca_passes_dead')
    takeons_gca = get_dat(data_gca, 'gca_take_ons')
    shots_gca = get_dat(data_gca, 'gca_shots')
    fouls_gca = get_dat(data_gca, 'gca_fouled')
    defense_gca = get_dat(data_gca, 'gca_defense')
    
    
    ##### defensive data ######
    #tackles
    tackles_won = get_dat(data_defense,'tackles_won')
    tackles_def = get_dat(data_defense, 'tackles_def_3rd')
    tackles_mid = get_dat(data_defense, 'tackles_mid_3rd')
    tackles_att = get_dat(data_defense, 'tackles_att_3rd')

    #challenges 
    dribblers_tackled = get_dat(data_defense, 'challenge_tackles')
    dribbles_challenged = get_dat(data_defense, 'challenges')
    challenges_lost = get_dat(data_defense, 'challenges_lost')

    #blocks 
    shots_blocked = get_dat(data_defense, 'blocked_shots')
    pass_blocked = get_dat(data_defense, 'blocked_passes')
    clearances = get_dat(data_defense, 'clearances')
    def_errors = get_dat(data_defense, 'errors')
    
    
    ##### possession data #####
    #touches 
    def_penarea_touches = get_dat(data_possession, 'touches_def_pen_area')
    def_third_touches = get_dat(data_possession, 'touches_def_3rd')
    mid_third_touches = get_dat(data_possession, 'touches_mid_3rd')
    att_third_touches = get_dat(data_possession, 'touches_att_3rd')
    pen_area_touches = get_dat(data_possession, 'touches_att_pen_area')

    #carries
    carry_distance = get_dat(data_possession, 'carries_distance')
    prg_carry_distance = get_dat(data_possession, 'carries_progressive_distance')
    final_third_carries = get_dat(data_possession, 'carries_into_final_third')
    pen_area_carries = get_dat(data_possession, 'carries_into_penalty_area')
    miscontrols = get_dat(data_possession, 'miscontrols')
    dispossesed = get_dat(data_possession, 'dispossessed')

    #receiving 
    pass_received = get_dat(data_possession, 'passes_received')
    prg_pass_received = get_dat(data_possession, 'progressive_passes_received')


    ###########################################################
    #creating dataframe
    df = pd.DataFrame({
        'Date': date,
        'Day': day,
        'Matchweek': matchweek,
        'Venue': venue,
        'Result': result,
        'Team': team,
        'Opponent': opponent, 
        'Start': start, 
        'Position': position,
        'Minutes Played': mins, 
        'Goals': goals, 
        'Assists': assist, 
        'Penalties Scored': pen_goals, 
        'Penalties Attempted': pen_attempted, 
        'Shots': shots, 
        'Shots on Target': sot, 
        'Yellow Cards': yellow, 
        'Red Cards': red, 
        'Touches': touches, 
        'Tackles': tackles,
        'Interceptions': interceptions, 
        'Blocks': blocks, 
        'xG': xg,
        'npxG': npxg,
        'xAG': xag, 
        'Shot Creating Actions': sca, 
        'Goal Creating Actions': gca, 
        'Passes Completed': passes_completed, 
        'Passes Attempted': passes_attempted, 
        'Progressive Passes': prg_passes,
        'Carries': carries, 
        'Progressive Carries': prg_carries, 
        'Take-ons Attempted': takeon_att, 
        'Successful Take-ons': takeon_suc,
        
        'Passing Distance': pass_distance, 
        'Progressive Passing Distance': prg_pass_distance,
        'Short Passes Completed' : short_pass_completed,
        'Short Passes Attempted': short_pass_attempted,
        'Medium Passes Completed': med_pass_completed,
        'Medium Passes Attempted': med_pass_attempted,
        'Long Passes Completed': long_pass_completed,
        'Long Passes Attempted': long_pass_attempted,
        'Expected Assists': xa,
        'Key Passes': keypass,
        'Passes into Final Third': finalthird_pass,
        'Passes into Penalty Area': penaltyarea_pass,
        'Crosses into Penalty Area': penaltyarea_cross,
    
        'Live Pass': live_pass,
        'Dead Pass': dead_pass,
        'Free Kick Pass': freekick_pass,
        'Through Balls': through_balls,
        'Switches': switches,
        'Crosses': crosses,
        'Throw Ins Taken': throwin,
        'Corners Taken': corners,
        'Passes Offside': offside_pass,
    
        'Live SCA': live_sca,
        'Deadball SCA': deadball_sca,
        'Take-on SCA': takeons_sca,
        'Shot SCA': shots_sca,
        'Foul SCA': fouls_sca,
        'Defense SCA': defense_sca,
    
        'Live GCA': live_gca,
        'Deadball GCA': deadball_gca,
        'Take-on GCA': takeons_gca,
        'Shot GCA': shots_gca,
        'Foul GCA': fouls_gca,
        'Defense GCA': defense_gca,
    
        'Tackles Won': tackles_won,
        'Defensive Third Tackles': tackles_def,
        'Middle Third Tackles': tackles_mid,
        'Attacking Third Tackles': tackles_att,
        'Dribblers Tackled': dribblers_tackled,
        'Dribblers Tackled Attempts': dribbles_challenged,
        'Challenges Lost': challenges_lost,
        'Shots Blocked': shots_blocked,
        'Passes Blocked': pass_blocked,
        'Clearances': clearances,
        'Defensive Errors': def_errors,
    
        'Defensive Penalty Area Touches': def_penarea_touches,
        'Defensive Third Touches': def_third_touches,
        'Middle Third Touches': mid_third_touches,
        'Attacking Third Touches': att_third_touches,
        'Penalty Area Touches': pen_area_touches,
        'Carry Distance': carry_distance,
        'Progressive Carry Distance': prg_carry_distance,
        'Final Third Carries': final_third_carries,
        'Carries into Penalty Area': pen_area_carries,
        'Miscontrols': miscontrols,
        'Dispossessed': dispossesed,
        'Passes Received': pass_received,
        'Progressive Passes Received': prg_pass_received
    })
    
    for column in df.columns:
        df[column] = df[column].replace('',0) #replacing empty strings with zero 
    
    #modifying data types
    df = df.astype({
        'Date': 'datetime64[ns]',
        'Day': 'object',
        'Matchweek': 'int64',
        'Venue': 'object',
        'Result': 'object',
        'Team': 'object',
        'Opponent': 'object', 
        'Start': 'object', 
        'Position': 'object',
        'Minutes Played': 'int64', 
        'Goals': 'int64', 
        'Assists': 'int64', 
        'Penalties Scored': 'int64', 
        'Penalties Attempted': 'int64', 
        'Shots': 'int64', 
        'Shots on Target': 'int64', 
        'Yellow Cards': 'int64', 
        'Red Cards': 'int64', 
        'Touches': 'int64', 
        'Tackles': 'int64',
        'Interceptions': 'int64', 
        'Blocks': 'int64', 
        'xG': 'float64',
        'npxG': 'float64',
        'xAG': 'float64', 
        'Shot Creating Actions': 'int64', 
        'Goal Creating Actions': 'int64', 
        'Passes Completed': 'int64', 
        'Passes Attempted': 'int64', 
        'Progressive Passes': 'int64',
        'Carries': 'int64', 
        'Progressive Carries': 'int64', 
        'Take-ons Attempted': 'int64', 
        'Successful Take-ons': 'int64',
        
        'Short Passes Completed' : 'int64',
        'Short Passes Attempted': 'int64',
        'Medium Passes Completed': 'int64',
        'Medium Passes Attempted': 'int64',
        'Long Passes Completed': 'int64',
        'Long Passes Attempted': 'int64',
        'Expected Assists': 'float64',
        'Key Passes': 'int64',
        'Passes into Final Third': 'int64',
        'Passes into Penalty Area': 'int64',
        'Crosses into Penalty Area': 'int64',
    
        'Live Pass': 'int64',
        'Dead Pass': 'int64',
        'Free Kick Pass': 'int64',
        'Through Balls': 'int64',
        'Switches': 'int64',
        'Crosses': 'int64',
        'Throw Ins Taken': 'int64',
        'Corners Taken': 'int64',
        'Passes Offside': 'int64',
    
        'Live SCA': 'int64',
        'Deadball SCA': 'int64',
        'Take-on SCA': 'int64',
        'Shot SCA': 'int64',
        'Foul SCA': 'int64',
        'Defense SCA': 'int64',
    
        'Live GCA': 'int64',
        'Deadball GCA': 'int64',
        'Take-on GCA': 'int64',
        'Shot GCA': 'int64',
        'Foul GCA': 'int64',
        'Defense GCA': 'int64',
    
        'Tackles Won': 'int64',
        'Defensive Third Tackles': 'int64',
        'Middle Third Tackles': 'int64',
        'Attacking Third Tackles': 'int64',
        'Dribblers Tackled': 'int64',
        'Dribblers Tackled Attempts': 'int64',
        'Challenges Lost': 'int64',
        'Shots Blocked': 'int64',
        'Passes Blocked': 'int64',
        'Clearances': 'int64',
        'Defensive Errors': 'int64',
    
        'Defensive Penalty Area Touches': 'int64',
        'Defensive Third Touches': 'int64',
        'Middle Third Touches': 'int64',
        'Attacking Third Touches': 'int64',
        'Penalty Area Touches': 'int64',
        'Carry Distance': 'float64',
        'Progressive Carry Distance': 'float64',
        'Final Third Carries': 'int64',
        'Carries into Penalty Area': 'int64',
        'Miscontrols': 'int64',
        'Dispossessed': 'int64',
        'Passes Received': 'int64',
        'Progressive Passes Received': 'int64'
    })
    
    #changing date column data type 
    df['Date'] = df['Date'].dt.date
    
    ###############################################################################
    
    #now we want to get the fpl data 
    start_year = year_range[:4] #getting the whole of the first year 
    end_year = year_range[-2:]
    format_year = f'{start_year}-{end_year}'
    directory = f'Fantasy-Premier-League/data/{format_year}/players'
    files = os.listdir(directory)
    csv_file = [f for f in files if fpl_name in f]
    fpldf = pd.read_csv(os.path.join(directory, csv_file[0]) + '/gw.csv')

    #some of the columns are duplicated (from the webscraped data), so we want to remove these 
    fpldf = fpldf.drop(['assists', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'fixture',
                                          'goals_conceded', 'goals_scored', 'penalties_missed', 'penalties_saved', 'red_cards'
                                          ,'team_a_score', 'team_h_score', 'was_home', 'yellow_cards', 'element', 'opponent_team', 'starts'
                                          ,'expected_goals_conceded'], axis=1, 
                       errors = 'ignore')

    #we now want to change the 'kickoff time' feature into a datetime object 
    fpldf['kickoff_time'] = pd.to_datetime(fpldf['kickoff_time'])

    #creating new column, which is just the kickoff date 
    fpldf['kickoff_date'] = fpldf['kickoff_time'].dt.date
    
    finaldf = pd.merge(df, fpldf, left_on='Date', right_on='kickoff_date', how='inner')
    
    return(finaldf)

get_premgames function

In [8]:
def get_premgames (code, player):
    base_url = f'https://fbref.com/en/players/{code}/{player}'
    html = requests.get(base_url).text
    soup = BeautifulSoup(html, 'lxml')
    summary = soup.find('table', class_='stats_table sortable min_width')
    table = summary.find('tbody')
    
    comp = table.find_all('td', attrs={'data-stat': 'comp_level'})
    comp_text = [cell.get_text() for cell in comp]
    games = table.find_all('td', attrs={'data-stat': 'games'})
    games_text = [cell.get_text() for cell in games]
    season = table.find_all('th', attrs={'data-stat': 'year_id'})
    season_text  = [cell.get_text() for cell in season]
    
    df = pd.DataFrame({
        'Season': season_text,
        'Competition': comp_text, 
        'Games Played': games_text
    })

    season_list = ('2023-2024', '2022-2023', '2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018')
    total_games = df[(df['Competition'] == '1. Premier League') & (df['Season'].isin(season_list))]['Games Played']
    total_games = pd.to_numeric(total_games)
    prem_games = total_games.sum()
    return(prem_games)

This code specifies the location to place the .csv file into

In [10]:
nest_folder = ("Player_Data")
os.makedirs(nest_folder, exist_ok = True)

nest_folder_def = os.path.join("Player_Data", "Defenders")

## Now, we actually use the functions above to get the necessary data 

### Diogo Jota

In [35]:
jota_2324_finaldf = get_data('178ae8f8', '2023-2024', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [9]:
jota_2223_finaldf = get_data('178ae8f8', '2022-2023', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [10]:
jota_2122_finaldf = get_data('178ae8f8', '2021-2022', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [12]:
jota_2021_finaldf = get_data('178ae8f8', '2020-2021', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [6]:
jota_1920_finaldf = get_data('178ae8f8', '2019-2020', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [7]:
jota_1819_finaldf = get_data('178ae8f8', '2018-2019', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [26]:
jota_finaldat = pd.concat([jota_1819_finaldf, jota_1920_finaldf, jota_2021_finaldf, jota_2122_finaldf
                            , jota_2223_finaldf, jota_2324_finaldf], join = 'inner', ignore_index = True)
jota_finaldat.to_csv(os.path.join(nest_folder, "jota_finaldat.csv"))

### Luis Diaz

In [27]:
diaz_2324_finaldf = get_data('4a1a9578', '2023-2024', 'Luis-Diaz',  "Luis_Díaz")

In [28]:
diaz_2223_finaldf = get_data('4a1a9578', '2022-2023', 'Luis-Diaz',  "Luis_Díaz")

In [29]:
diaz_2122_finaldf = get_data('4a1a9578', '2021-2022', 'Luis-Diaz',  "Luis_Díaz")

In [30]:
diaz_finaldat = pd.concat([diaz_2122_finaldf, diaz_2223_finaldf, diaz_2324_finaldf], join = 'inner', ignore_index = True)
diaz_finaldat.to_csv(os.path.join(nest_folder, "diaz_finaldat.csv"))

### Cody Gakpo

In [31]:
gakpo_2324_finaldf = get_data('1971591f', '2023-2024', 'Cody-Gakpo',  "Cody_Gakpo")

In [32]:
gakpo_2223_finaldf = get_data('1971591f', '2022-2023', 'Cody-Gakpo',  "Cody_Gakpo")

In [33]:
gakpo_finaldat = pd.concat([gakpo_2223_finaldf, gakpo_2324_finaldf], join = 'inner', ignore_index = True)
gakpo_finaldat.to_csv(os.path.join(nest_folder, "gakpo_finaldat.csv"))

### Darwin Nunez

In [36]:
darwin_2324_finaldf = get_data('4d77b365', '2023-2024', 'Darwin-Nunez',  "Darwin_Núñez Ribeiro")

In [38]:
darwin_2223_finaldf = get_data('4d77b365', '2022-2023', 'Darwin-Nunez',  "Darwin")

In [39]:
darwin_finaldat = pd.concat([darwin_2223_finaldf, darwin_2324_finaldf], join = 'inner', ignore_index = True)
darwin_finaldat.to_csv(os.path.join(nest_folder, "darwin_finaldat.csv"))

# Updated Function
This new function compile_dat allows us to get all of the data for a single player in one function, instead of having to do it one by one as can be seen above. 

In [9]:
def compile_dat (code, player, fpl_name, checkgames):
    #these are the seasons for which we have fpl data 
    year_list = ('2023-24', '2022-23', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18')
    
    #active_years is a list that will store the seasons that a given player was active. The for loop will check to see which years a player was active
    #in the prem, and will append this year to the list
    active_years = []
    for year in year_list: 
        directory = f'Fantasy-Premier-League/data/{year}/players'
        files = os.listdir(directory)
        csv_file = [f for f in files if fpl_name in f]
        if csv_file:
            update_year = year[:5] + '20' + year[5:]
            active_years.append(update_year)
    
    #create an empty dictionary to store all of the dataframes 
    dataframes = {}
    
    #for each season a player was active, the get_data function from above is uesd to add the data for that season to the dataframes dictionary
    for year in active_years: 
        dataframes[year] = get_data_final(code, year, player, fpl_name)
        #1 minute pause in between each iteration, to ensure that we don't get banned from FBref
        time.sleep(60)
    
    #concatenate all of the dataframes, and return one final concatenated dataframe
    finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)
    
    if checkgames:
        games_played = get_premgames(code, player)
        if games_played == finaldf.shape[0]:
            return(finaldf)
        else:
            return(print('The number of Premier League games played by ' + player + '(' + str(games_played) +  ') does not match the number of rows in the dataframe ('+ str(finaldf.shape[0]) + ')'))
    else: 
        return(finaldf)

#note that the to_csv code is not included in this function, because there are certain players whose names were changed in between seasons 
#(e.g. Diogo Teixeira da Silva to Diogo Jota). In these cases, we need to use the compile function twice to get all of the data, before concatenating
#and then finally exporting to csv 

## Arsenal

In [43]:
saka_finaldat = compile_dat('bc7dc64d', 'Bukayo-Saka', 'Bukayo')
saka_finaldat.to_csv(os.path.join(nest_folder, "saka_finaldat.csv"))

In [45]:
havertz_finaldat = compile_dat('fed7cb61', 'Kai-Havertz', 'Kai_Havertz')
havertz_finaldat.to_csv(os.path.join(nest_folder, "havertz_finaldat.csv"))

In [50]:
jesus_finaldat = compile_dat('b66315ae', 'Gabriel-Jesus', 'de Jesus')
jesus_finaldat.to_csv(os.path.join(nest_folder, "jesus_finaldat.csv"))

In [47]:
trossard_finaldat = compile_dat('38ceb24a', 'Leandro-Trossard', 'Leandro_Trossard')
trossard_finaldat.to_csv(os.path.join(nest_folder, "trossard_finaldat.csv"))

In [51]:
martinelli_finaldat = compile_dat('48a5a5d6', 'Gabriel-Martinelli', 'Martinelli')
martinelli_finaldat.to_csv(os.path.join(nest_folder, "martinelli_finaldat.csv"))

In [52]:
odegaard_finaldat = compile_dat('79300479', 'Martin-Odegaard', 'Ødegaard')
odegaard_finaldat.to_csv(os.path.join(nest_folder, "odegaard_finaldat.csv"))

In [53]:
sterling_finaldat = compile_dat('b400bde0', 'Raheem-Sterling', 'Raheem_Sterling')
sterling_finaldat.to_csv(os.path.join(nest_folder, "sterling_finaldat.csv"))

In [88]:
jorginho_finaldat = compile_dat('45db685d', 'Jorginho', 'Jorge Luiz_Frello', checkgames = True)
if isinstance(jorginho_finaldat, pd.DataFrame):
    jorginho_finaldat.to_csv(os.path.join(nest_folder, "jorginho_finaldat.csv"))

In [100]:
rice_finaldat = compile_dat('1c7012b8', 'Declan-Rice', 'Declan_Rice', checkgames = True)
if isinstance(rice_finaldat, pd.DataFrame):
    rice_finaldat.to_csv(os.path.join(nest_folder, "rice_finaldat.csv"))

In [90]:
partey_finaldat = compile_dat('529f49ab', 'Thomas-Partey', 'Thomas_Partey', checkgames = True)
if isinstance(partey_finaldat, pd.DataFrame):
    partey_finaldat.to_csv(os.path.join(nest_folder, "partey_finaldat.csv"))

Arsenal Defenders

In [11]:
gabriel_finaldat = compile_dat('67ac5bb8', 'Gabriel-Magalhaes', 'Magalh', checkgames = True)
if isinstance(gabriel_finaldat, pd.DataFrame):
    gabriel_finaldat.to_csv(os.path.join(nest_folder_def, "gabriel_finaldat.csv"))

In [12]:
saliba_finaldat = compile_dat('972aeb2a', 'William-Saliba', 'William_Saliba', checkgames = True)
if isinstance(saliba_finaldat, pd.DataFrame):
    saliba_finaldat.to_csv(os.path.join(nest_folder_def, "saliba_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [13]:
white_finaldat = compile_dat('35e413f1', 'Ben-White', 'Ben_White', checkgames = True)
if isinstance(white_finaldat, pd.DataFrame):
    white_finaldat.to_csv(os.path.join(nest_folder_def, "white_finaldat.csv"))

In [14]:
zinchenko_finaldat = compile_dat('51cf8561', 'Oleksandr-Zinchenko', 'Oleksandr_Zinchenko', checkgames = True)
if isinstance(zinchenko_finaldat, pd.DataFrame):
    zinchenko_finaldat.to_csv(os.path.join(nest_folder_def, "zinchenko_finaldat.csv"))

In [15]:
kiwior_finaldat = compile_dat('dc3e663e', 'Jakub-Kiwior', 'Jakub_Kiwior', checkgames = True)
if isinstance(kiwior_finaldat, pd.DataFrame):
    kiwior_finaldat.to_csv(os.path.join(nest_folder_def, "kiwior_finaldat.csv"))

In [16]:
tierney_finaldat = compile_dat('fce2302c', 'Kieran-Tierney', 'Kieran_Tierney', checkgames = True)
if isinstance(tierney_finaldat, pd.DataFrame):
    tierney_finaldat.to_csv(os.path.join(nest_folder_def, "tierney_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [17]:
tomiyasu_finaldat = compile_dat('b3af9be1', 'Takehiro-Tomiyasu', 'Takehiro_Tomiyasu', checkgames = True)
if isinstance(tomiyasu_finaldat, pd.DataFrame):
    tomiyasu_finaldat.to_csv(os.path.join(nest_folder_def, "tomiyasu_finaldat.csv"))

## Aston Villa

In [61]:
duran_finaldat = compile_dat('414184f7', 'Jhon-Duran', 'Jhon', checkgames=True)
duran_finaldat.to_csv(os.path.join(nest_folder, "duran_finaldat.csv"))

In [63]:
watkins_finaldat = compile_dat('aed3a70f', 'Ollie-Watkins', 'Ollie_Watkins', checkgames=True)
if isinstance(watkins_finaldat, pd.DataFrame):
    watkins_finaldat.to_csv(os.path.join(nest_folder, "watkins_finaldat.csv"))

In [64]:
ramsey_finaldat = compile_dat('1544f145', 'Jacob-Ramsey', 'Jacob_Ramsey', checkgames = True)
if isinstance(ramsey_finaldat, pd.DataFrame):
    ramsey_finaldat.to_csv(os.path.join(nest_folder, "ramsey_finaldat.csv"))

In [68]:
mcginn_finaldat = compile_dat('90f91999', 'John-McGinn', 'John_McGinn', checkgames = False)
if isinstance(mcginn_finaldat, pd.DataFrame):
    mcginn_finaldat.to_csv(os.path.join(nest_folder, "mcginn_finaldat.csv"))

In [66]:
bailey_finaldat = compile_dat('3a233281', 'Leon-Bailey', 'Leon_Bailey', checkgames = True)
if isinstance(bailey_finaldat, pd.DataFrame):
    bailey_finaldat.to_csv(os.path.join(nest_folder, "bailey_finaldat.csv"))

In [84]:
buendia_finaldat = compile_dat('66b76d44', 'Emi-Buendia', 'Emiliano_Buen', checkgames = True)
if isinstance(buendia_finaldat, pd.DataFrame):
    buendia_finaldat.to_csv(os.path.join(nest_folder, "buendia_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/2405533476.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [69]:
tielemans_finaldat = compile_dat('56f7a928', 'Youri-Tielemans', 'Youri_Tielemans', checkgames = True)
if isinstance(tielemans_finaldat, pd.DataFrame):
    tielemans_finaldat.to_csv(os.path.join(nest_folder, "tielemans_finaldat.csv"))

In [92]:
barkley_finaldat = compile_dat('3a24769f', 'Ross-Barkley', 'Ross_Barkley', checkgames = False)
if isinstance(barkley_finaldat, pd.DataFrame):
    barkley_finaldat.to_csv(os.path.join(nest_folder, "barkley_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/2561086557.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [86]:
onana_finaldat = compile_dat('828657ff', 'Amadou-Onana', 'Amadou_Onana', checkgames = True)
if isinstance(onana_finaldat, pd.DataFrame):
    onana_finaldat.to_csv(os.path.join(nest_folder, "onana_finaldat.csv"))

In [87]:
rogers_finaldat = compile_dat('2e5915f1', 'Morgan-Rogers', 'Morgan_Rogers', checkgames = True)
if isinstance(rogers_finaldat, pd.DataFrame):
    rogers_finaldat.to_csv(os.path.join(nest_folder, "rogers_finaldat.csv"))

Aston Villa Defenders

In [18]:
digne_finaldat = compile_dat('1b84dbe1', 'Lucas-Digne', 'Lucas_Digne', checkgames = True)
if isinstance(digne_finaldat, pd.DataFrame):
    digne_finaldat.to_csv(os.path.join(nest_folder_def, "digne_finaldat.csv"))

In [19]:
konsa_finaldat = compile_dat('0313a347', 'Ezri-Konsa', 'Ezri_Konsa', checkgames = True)
if isinstance(konsa_finaldat, pd.DataFrame):
    konsa_finaldat.to_csv(os.path.join(nest_folder_def, "konsa_finaldat.csv"))

In [20]:
pau_finaldat = compile_dat('532e1e4f', 'Pau-Torres', 'Pau_Torres', checkgames = True)
if isinstance(pau_finaldat, pd.DataFrame):
    pau_finaldat.to_csv(os.path.join(nest_folder_def, "pau_finaldat.csv"))

In [21]:
cash_finaldat = compile_dat('2389cdc2', 'Matty-Cash', 'Cash', checkgames = True)
if isinstance(cash_finaldat, pd.DataFrame):
    cash_finaldat.to_csv(os.path.join(nest_folder_def, "cash_finaldat.csv"))

In [22]:
dcarlos_finaldat = compile_dat('b4a014b1', 'Diego-Carlos', 'Diego Carlos', checkgames = True)
if isinstance(dcarlos_finaldat, pd.DataFrame):
    dcarlos_finaldat.to_csv(os.path.join(nest_folder_def, "dcarlos_finaldat.csv"))

In [23]:
hause_finaldat = compile_dat('bf8fad51', 'Kortney-Hause', 'Kortney_Hause', checkgames = True)
if isinstance(hause_finaldat, pd.DataFrame):
    hause_finaldat.to_csv(os.path.join(nest_folder_def, "hause_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [24]:
mings_finaldat = compile_dat('8397a50c', 'Tyrone-Mings', 'Tyrone_Mings', checkgames = True)
if isinstance(mings_finaldat, pd.DataFrame):
    mings_finaldat.to_csv(os.path.join(nest_folder_def, "mings_finaldat.csv"))

## Bournemouth

In [101]:
unal_finaldat = compile_dat('f8eca1b6', 'Enes-Unal', 'Enes', checkgames = True)
if isinstance(unal_finaldat, pd.DataFrame):
    unal_finaldat.to_csv(os.path.join(nest_folder, "unal_finaldat.csv"))

In [102]:
semenyo_finaldat = compile_dat('efd2ec23', 'Antoine-Semenyo', 'Antoine_Semenyo', checkgames = True)
if isinstance(semenyo_finaldat, pd.DataFrame):
    semenyo_finaldat.to_csv(os.path.join(nest_folder, "semenyo_finaldat.csv"))

In [103]:
sinisterra_finaldat = compile_dat('8e16dd48', 'Luis-Sinisterra', 'Luis_Sinisterra', checkgames = True)
if isinstance(sinisterra_finaldat, pd.DataFrame):
    sinisterra_finaldat.to_csv(os.path.join(nest_folder, "sinisterra_finaldat.csv"))

In [104]:
tavernier_finaldat = compile_dat('5c0da4a4', 'Marcus-Tavernier', 'Marcus_Tavernier', checkgames = True)
if isinstance(tavernier_finaldat, pd.DataFrame):
    tavernier_finaldat.to_csv(os.path.join(nest_folder, "tavernier_finaldat.csv"))

In [105]:
cook_finaldat = compile_dat('2afc7272', 'Lewis-Cook', 'Lewis_Cook', checkgames = True)
if isinstance(cook_finaldat, pd.DataFrame):
    cook_finaldat.to_csv(os.path.join(nest_folder, "cook_finaldat.csv"))

In [106]:
kluivert_finaldat = compile_dat('4c3a6744', 'Justin-Kluivert', 'Justin_Kluivert', checkgames = True)
if isinstance(kluivert_finaldat, pd.DataFrame):
    kluivert_finaldat.to_csv(os.path.join(nest_folder, "kluivert_finaldat.csv"))

In [107]:
dango_finaldat = compile_dat('2f9e4435', 'Dango-Ouattara', 'Dango_Ouattara', checkgames = True)
if isinstance(dango_finaldat, pd.DataFrame):
    dango_finaldat.to_csv(os.path.join(nest_folder, "dango_finaldat.csv"))

In [108]:
christie_finaldat = compile_dat('26ce2263', 'Ryan-Christie', 'Ryan_Christie', checkgames = True)
if isinstance(christie_finaldat, pd.DataFrame):
    christie_finaldat.to_csv(os.path.join(nest_folder, "christie_finaldat.csv"))

In [109]:
scott_finaldat = compile_dat('104d0bb8', 'Alex-Scott', 'Alex_Scott', checkgames = True)
if isinstance(scott_finaldat, pd.DataFrame):
    scott_finaldat.to_csv(os.path.join(nest_folder, "scott_finaldat.csv"))

In [110]:
billing_finaldat = compile_dat('d328a254', 'Philip-Billing', 'Philip_Billing', checkgames = True)
if isinstance(billing_finaldat, pd.DataFrame):
    billing_finaldat.to_csv(os.path.join(nest_folder, "billing_finaldat.csv"))

In [111]:
adams_finaldat = compile_dat('2b09d998', 'Tyler-Adams', 'Tyler_Adams', checkgames = True)
if isinstance(adams_finaldat, pd.DataFrame):
    adams_finaldat.to_csv(os.path.join(nest_folder, "adams_finaldat.csv"))

In [113]:
brooks_finaldat = compile_dat('dc4cae05', 'David-Brooks', 'David_Brooks', checkgames = True)
if isinstance(brooks_finaldat, pd.DataFrame):
    brooks_finaldat.to_csv(os.path.join(nest_folder, "brooks_finaldat.csv"))

Bournemouth Defenders

In [25]:
zabarnyi_finaldat = compile_dat('88968486', 'Illia_Zabarnyi', 'Illia_Zabarnyi', checkgames = False)
if isinstance(zabarnyi_finaldat, pd.DataFrame):
    zabarnyi_finaldat.to_csv(os.path.join(nest_folder_def, "zabarnyi_finaldat.csv"))

The number of Premier League games played by Illia_Zabarnyi(92) does not match the number of rows in the dataframe (42)


In [26]:
kerkez_finaldat = compile_dat('0ad53bdc', 'Milos-Kerkez', 'Milos_Kerkez', checkgames = True)
if isinstance(kerkez_finaldat, pd.DataFrame):
    kerkez_finaldat.to_csv(os.path.join(nest_folder_def, "kerkez_finaldat.csv"))

In [27]:
asmith_finaldat = compile_dat('20b104bc', 'Adam-Smith', 'Adam_Smith', checkgames = True)
if isinstance(asmith_finaldat, pd.DataFrame):
    asmith_finaldat.to_csv(os.path.join(nest_folder_def, "asmith_finaldat.csv"))

In [28]:
senesi_finaldat = compile_dat('35141f4c', 'Marcos-Senesi', 'Senesi', checkgames = True)
if isinstance(senesi_finaldat, pd.DataFrame):
    senesi_finaldat.to_csv(os.path.join(nest_folder_def, "senesi_finaldat.csv"))

In [31]:
aarons_finaldat = compile_dat('774cf58b', 'Max-Aarons', 'Max_Aarons', checkgames = True)
if isinstance(aarons_finaldat, pd.DataFrame):
    aarons_finaldat.to_csv(os.path.join(nest_folder_def, "aarons_finaldat.csv"))

## Brentford

In [114]:
toney_finaldat = compile_dat('e09f279b', 'Ivan-Toney', 'Ivan_Toney', checkgames = True)
if isinstance(toney_finaldat, pd.DataFrame):
    toney_finaldat.to_csv(os.path.join(nest_folder, "toney_finaldat.csv"))

In [115]:
wissa_finaldat = compile_dat('2500cef9', 'Yoane-Wissa', 'Yoane_Wissa', checkgames = True)
if isinstance(wissa_finaldat, pd.DataFrame):
    wissa_finaldat.to_csv(os.path.join(nest_folder, "wissa_finaldat.csv"))

In [116]:
mbeumo_finaldat = compile_dat('6afaebf2', 'Bryan-Mbeumo', 'Bryan_Mbeumo', checkgames = True)
if isinstance(mbeumo_finaldat, pd.DataFrame):
    mbeumo_finaldat.to_csv(os.path.join(nest_folder, "mbeumo_finaldat.csv"))

In [117]:
jensen_finaldat = compile_dat('0f134faf', 'Mathias-Jensen', 'Mathias_Jensen', checkgames = True)
if isinstance(jensen_finaldat, pd.DataFrame):
    jensen_finaldat.to_csv(os.path.join(nest_folder, "jensen_finaldat.csv"))

In [118]:
schade_finaldat = compile_dat('52afb588', 'Kevin-Schade', 'Kevin_Schade', checkgames = True)
if isinstance(schade_finaldat, pd.DataFrame):
    schade_finaldat.to_csv(os.path.join(nest_folder, "schade_finaldat.csv"))

In [119]:
janelt_finaldat = compile_dat('8449d35e', 'Vitaly-Janelt', 'Vitaly_Janelt', checkgames = True)
if isinstance(janelt_finaldat, pd.DataFrame):
    janelt_finaldat.to_csv(os.path.join(nest_folder, "janelt_finaldat.csv"))

In [120]:
damsgaard_finaldat = compile_dat('215f3907', 'Mikkel-Damsgaard', 'Mikkel_Damsgaard', checkgames = True)
if isinstance(damsgaard_finaldat, pd.DataFrame):
    damsgaard_finaldat.to_csv(os.path.join(nest_folder, "damsgaard_finaldat.csv"))

In [121]:
norgaard_finaldat = compile_dat('df0a4c90', 'Christian-Norgaard', 'Christian_Nø', checkgames = True)
if isinstance(norgaard_finaldat, pd.DataFrame):
    norgaard_finaldat.to_csv(os.path.join(nest_folder, "norgaard_finaldat.csv"))

In [122]:
lewispotter_finaldat = compile_dat('41f08ac8', 'Keane-Lewis-Potter', 'Keane_Lewis', checkgames = True)
if isinstance(lewispotter_finaldat, pd.DataFrame):
    lewispotter_finaldat.to_csv(os.path.join(nest_folder, "lewispotter_finaldat.csv"))

Brentford Defenders

In [32]:
collins_finaldat = compile_dat('a8c19eb8', 'Nathan-Collins', 'Nathan_Collins', checkgames = True)
if isinstance(collins_finaldat, pd.DataFrame):
    collins_finaldat.to_csv(os.path.join(nest_folder_def, "collins_finaldat.csv"))

In [33]:
ajer_finaldat = compile_dat('a8c0acb7', 'Kristoffer-Ajer', 'Kristoffer_Ajer', checkgames = True)
if isinstance(ajer_finaldat, pd.DataFrame):
    ajer_finaldat.to_csv(os.path.join(nest_folder_def, "ajer_finaldat.csv"))

In [34]:
roerslev_finaldat = compile_dat('57c94db2', 'Mads-Roerslev', 'Mads_Roerslev', checkgames = True)
if isinstance(roerslev_finaldat, pd.DataFrame):
    roerslev_finaldat.to_csv(os.path.join(nest_folder_def, "roerslev_finaldat.csv"))

In [35]:
pinnock_finaldat = compile_dat('e541326e', 'Ethan-Pinnock', 'Ethan_Pinnock', checkgames = True)
if isinstance(pinnock_finaldat, pd.DataFrame):
    pinnock_finaldat.to_csv(os.path.join(nest_folder_def, "pinnock_finaldat.csv"))

In [36]:
mee_finaldat = compile_dat('8df7a2fb', 'Ben-Mee', 'Ben_Mee', checkgames = True)
if isinstance(mee_finaldat, pd.DataFrame):
    mee_finaldat.to_csv(os.path.join(nest_folder_def, "mee_finaldat.csv"))

In [37]:
rhenry_finaldat = compile_dat('8a3c1dc7', 'Rico-Henry', 'Rico_Henry', checkgames = True)
if isinstance(rhenry_finaldat, pd.DataFrame):
    rhenry_finaldat.to_csv(os.path.join(nest_folder_def, "rhenry_finaldat.csv"))

In [38]:
hickey_finaldat = compile_dat('1780bb4a', 'Aaron-Hickey', 'Aaron_Hickey', checkgames = True)
if isinstance(hickey_finaldat, pd.DataFrame):
    hickey_finaldat.to_csv(os.path.join(nest_folder_def, "hickey_finaldat.csv"))

## Brighton

In [123]:
welbeck_finaldat = compile_dat('ce5143da', 'Danny-Welbeck', 'Danny_Welbeck', checkgames = True)
if isinstance(welbeck_finaldat, pd.DataFrame):
    welbeck_finaldat.to_csv(os.path.join(nest_folder, "welbeck_finaldat.csv"))

In [124]:
jpedro_finaldat = compile_dat('e8832875', 'Joao-Pedro', 'Junqueira', checkgames = True)
if isinstance(jpedro_finaldat, pd.DataFrame):
    jpedro_finaldat.to_csv(os.path.join(nest_folder, "jpedro_finaldat.csv"))

In [125]:
ferguson_finaldat = compile_dat('4596da74', 'Evan-Ferguson', 'Evan_Ferguson', checkgames = True)
if isinstance(ferguson_finaldat, pd.DataFrame):
    ferguson_finaldat.to_csv(os.path.join(nest_folder, "ferguson_finaldat.csv"))

In [126]:
mitoma_finaldat = compile_dat('74618572', 'Kaoru-Mitoma', 'Kaoru_Mitoma', checkgames = True)
if isinstance(mitoma_finaldat, pd.DataFrame):
    mitoma_finaldat.to_csv(os.path.join(nest_folder, "mitoma_finaldat.csv"))

In [127]:
adingra_finaldat = compile_dat('4dcec659', 'Simon-Adingra', 'Simon_Adingra', checkgames = True)
if isinstance(adingra_finaldat, pd.DataFrame):
    adingra_finaldat.to_csv(os.path.join(nest_folder, "adingra_finaldat.csv"))

In [128]:
hinshelwood_finaldat = compile_dat('e98211e7', 'Jack-Hinshelwood', 'Jack_Hinshelwood', checkgames = True)
if isinstance(hinshelwood_finaldat, pd.DataFrame):
    hinshelwood_finaldat.to_csv(os.path.join(nest_folder, "hinshelwood_finaldat.csv"))

In [129]:
milner_finaldat = compile_dat('2f90f6b8', 'James-Milner', 'James_Milner', checkgames = True)
if isinstance(milner_finaldat, pd.DataFrame):
    milner_finaldat.to_csv(os.path.join(nest_folder, "milner_finaldat.csv"))

In [130]:
enciso_finaldat = compile_dat('9cfbad36', 'Julio-Enciso', 'Julio_Enciso', checkgames = True)
if isinstance(enciso_finaldat, pd.DataFrame):
    enciso_finaldat.to_csv(os.path.join(nest_folder, "enciso_finaldat.csv"))

In [140]:
march_latedat = compile_dat('bb5fbd2b', 'Solly-March', 'Solly_March', checkgames = False)
march_earlydat = compile_dat('bb5fbd2b', 'Solly-March', 'Solomon_March', checkgames = False)
march_finaldat = pd.concat([march_latedat, march_earlydat], join = "inner", ignore_index = True)
if isinstance(march_finaldat, pd.DataFrame):
    march_finaldat.to_csv(os.path.join(nest_folder, "march_finaldat.csv"))

Brighton Defenders

In [98]:
veltman_finaldat = compile_dat('dad4b285', 'Joel-Veltman', 'Veltman', checkgames = True)
if isinstance(veltman_finaldat, pd.DataFrame):
    veltman_finaldat.to_csv(os.path.join(nest_folder_def, "veltman_finaldat.csv"))

In [40]:
dunk_finaldat = compile_dat('282f75f3', 'Lewis-Dunk', 'Lewis_Dunk', checkgames = True)
if isinstance(dunk_finaldat, pd.DataFrame):
    dunk_finaldat.to_csv(os.path.join(nest_folder_def, "dunk_finaldat.csv"))

In [41]:
vanhecke_finaldat = compile_dat('4fd08daa', 'Jan-Paul-van-Hecke', 'van Hecke', checkgames = True)
if isinstance(vanhecke_finaldat, pd.DataFrame):
    vanhecke_finaldat.to_csv(os.path.join(nest_folder_def, "vanhecke_finaldat.csv"))

In [42]:
estupinan_finaldat = compile_dat('d38fdf53', 'Pervis-Estupinan', 'Pervis_Estupi', checkgames = True)
if isinstance(estupinan_finaldat, pd.DataFrame):
    estupinan_finaldat.to_csv(os.path.join(nest_folder_def, "estupinan_finaldat.csv"))

In [43]:
webster_finaldat = compile_dat('c40b6180', 'Adam-Webster', 'Adam_Webster', checkgames = True)
if isinstance(webster_finaldat, pd.DataFrame):
    webster_finaldat.to_csv(os.path.join(nest_folder_def, "webster_finaldat.csv"))

In [44]:
igor_finaldat = compile_dat('a4e85758', 'Igor', 'Igor', checkgames = True)
if isinstance(igor_finaldat, pd.DataFrame):
    igor_finaldat.to_csv(os.path.join(nest_folder_def, "igor_finaldat.csv"))

In [45]:
lamptey_finaldat = compile_dat('f4e433d4', 'Tariq-Lamptey', 'Tariq_Lamptey', checkgames = True)
if isinstance(lamptey_finaldat, pd.DataFrame):
    lamptey_finaldat.to_csv(os.path.join(nest_folder_def, "lamptey_finaldat.csv"))

## Chelsea

In [142]:
jackson_finaldat = compile_dat('9c36ed83', 'Nicolas-Jackson', 'Nicolas_Jackson', checkgames = True)
if isinstance(jackson_finaldat, pd.DataFrame):
    jackson_finaldat.to_csv(os.path.join(nest_folder, "jackson_finaldat.csv"))

In [143]:
palmer_finaldat = compile_dat('dc7f8a28', 'Cole-Palmer', 'Cole_Palmer', checkgames = True)
if isinstance(palmer_finaldat, pd.DataFrame):
    palmer_finaldat.to_csv(os.path.join(nest_folder, "palmer_finaldat.csv"))

In [138]:
madueke_finaldat = compile_dat('bf34eebd', 'Noni-Madueke', 'Noni_Madueke', checkgames = True)
if isinstance(madueke_finaldat, pd.DataFrame):
    madueke_finaldat.to_csv(os.path.join(nest_folder, "madueke_finaldat.csv"))

In [139]:
felix_finaldat = compile_dat('8aafd64f', 'Joao-Felix', 'Sequeira', checkgames = True)
if isinstance(felix_finaldat, pd.DataFrame):
    felix_finaldat.to_csv(os.path.join(nest_folder, "felix_finaldat.csv"))

In [141]:
enzo_finaldat = compile_dat('5ff4ab71', 'Enzo-Fernandez', 'Enzo_Fern', checkgames = True)
if isinstance(enzo_finaldat, pd.DataFrame):
    enzo_finaldat.to_csv(os.path.join(nest_folder, "enzo_finaldat.csv"))

In [156]:
caicedo_finaldat = compile_dat('16264a81', 'Moises-Caicedo', 'Caicedo', checkgames = True)
if isinstance(caicedo_finaldat, pd.DataFrame):
    caicedo_finaldat.to_csv(os.path.join(nest_folder, "caicedo_finaldat.csv"))

In [145]:
neto_finaldat = compile_dat('7ba2eaa9', 'Pedro-Neto', 'Pedro_Lomba', checkgames = True)
if isinstance(neto_finaldat, pd.DataFrame):
    neto_finaldat.to_csv(os.path.join(nest_folder, "neto_finaldat.csv"))

In [146]:
dewsburyhall_finaldat = compile_dat('5c74c0f5', 'Kiernan-Dewsbury-Hall', 'Kiernan_Dewsbury', checkgames = True)
if isinstance(dewsburyhall_finaldat, pd.DataFrame):
    dewsburyhall_finaldat.to_csv(os.path.join(nest_folder, "dewsburyhall_finaldat.csv"))

In [162]:
mudryk_finaldat = compile_dat('049a888d', 'Mykhailo-Mudryk', 'Mykhailo_Mudryk', checkgames = False)
if isinstance(mudryk_finaldat, pd.DataFrame):
    mudryk_finaldat.to_csv(os.path.join(nest_folder, "mudryk_finaldat.csv"))

Chelsea Defenders

In [46]:
colwill_finaldat = compile_dat('700783e7', 'Levi-Colwill', 'Levi_Colwill', checkgames = True)
if isinstance(colwill_finaldat, pd.DataFrame):
    colwill_finaldat.to_csv(os.path.join(nest_folder_def, "colwill_finaldat.csv"))

In [47]:
cucurella_finaldat = compile_dat('1daec722', 'Marc-Cucurella', 'Marc_Cucurella', checkgames = True)
if isinstance(cucurella_finaldat, pd.DataFrame):
    cucurella_finaldat.to_csv(os.path.join(nest_folder_def, "cucurella_finaldat.csv"))

In [48]:
gusto_finaldat = compile_dat('d56b9520', 'Malo-Gusto', 'Malo_Gusto', checkgames = True)
if isinstance(gusto_finaldat, pd.DataFrame):
    gusto_finaldat.to_csv(os.path.join(nest_folder_def, "gusto_finaldat.csv"))

In [49]:
wfofana_finaldat = compile_dat('132a82f1', 'Wesley-Fofana', 'Wesley_Fofana', checkgames = True)
if isinstance(wfofana_finaldat, pd.DataFrame):
    wfofana_finaldat.to_csv(os.path.join(nest_folder_def, "wfofana_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [50]:
badiashile_finaldat = compile_dat('06df8256', 'Benoit-Badiashile', 'Badiashile', checkgames = True)
if isinstance(badiashile_finaldat, pd.DataFrame):
    badiashile_finaldat.to_csv(os.path.join(nest_folder_def, "badiashile_finaldat.csv"))

In [86]:
chilwell_finaldat = compile_dat('d2424d1b', 'Ben-Chilwell', 'Chilwell', checkgames = True)
if isinstance(chilwell_finaldat, pd.DataFrame):
    chilwell_finaldat.to_csv(os.path.join(nest_folder_def, "chilwell_finaldat.csv"))

In [52]:
disasi_finaldat = compile_dat('ad82197c', 'Axel-Disasi', 'Disasi', checkgames = True)
if isinstance(disasi_finaldat, pd.DataFrame):
    disasi_finaldat.to_csv(os.path.join(nest_folder_def, "disasi_finaldat.csv"))

In [53]:
rjames_finaldat = compile_dat('1265a93a', 'Reece-James', 'Reece_James', checkgames = True)
if isinstance(rjames_finaldat, pd.DataFrame):
    rjames_finaldat.to_csv(os.path.join(nest_folder_def, "rjames_finaldat.csv"))

In [54]:
tosin_finaldat = compile_dat('c81d773d', 'Tosin-Adarabioyo', 'Tosin_Adarabioyo', checkgames = True)
if isinstance(tosin_finaldat, pd.DataFrame):
    tosin_finaldat.to_csv(os.path.join(nest_folder_def, "tosin_finaldat.csv"))

## Crystal Palace

In [182]:
nketiah_latedat = compile_dat('a53649b7', 'Eddie-Nketiah', 'Eddie_Nketiah', checkgames = False)
nketiah_earlydat = compile_dat('a53649b7', 'Eddie-Nketiah', 'Edward_Nketiah', checkgames = False)
nketiah_finaldat = pd.concat([nketiah_latedat, nketiah_earlydat], join = "inner", ignore_index = True)
if isinstance(nketiah_finaldat, pd.DataFrame):
    nketiah_finaldat.to_csv(os.path.join(nest_folder, "nketiah_finaldat.csv"))

In [149]:
mateta_finaldat = compile_dat('50e6dc35', 'Jean-Philippe-Mateta', 'Jean-Philippe_Mateta', checkgames = True)
if isinstance(mateta_finaldat, pd.DataFrame):
    mateta_finaldat.to_csv(os.path.join(nest_folder, "mateta_finaldat.csv"))

In [150]:
eze_finaldat = compile_dat('ae4fc6a4', 'Eberechi-Eze', 'Eberechi_Eze', checkgames = True)
if isinstance(eze_finaldat, pd.DataFrame):
    eze_finaldat.to_csv(os.path.join(nest_folder, "eze_finaldat.csv"))

In [151]:
wharton_finaldat = compile_dat('4b542852', 'Adam-Wharton', 'Adam_Wharton', checkgames = True)
if isinstance(wharton_finaldat, pd.DataFrame):
    wharton_finaldat.to_csv(os.path.join(nest_folder, "wharton_finaldat.csv"))

In [152]:
cheick_finaldat = compile_dat('ce4f40c7', 'Cheick-Doucoure', 'Cheick_Doucou', checkgames = True)
if isinstance(cheick_finaldat, pd.DataFrame):
    cheick_finaldat.to_csv(os.path.join(nest_folder, "cheick_finaldat.csv"))

In [153]:
lerma_finaldat = compile_dat('9b5ce51a', 'Jefferson-Lerma', 'Jefferson_Lerma', checkgames = True)
if isinstance(lerma_finaldat, pd.DataFrame):
    lerma_finaldat.to_csv(os.path.join(nest_folder, "lerma_finaldat.csv"))

In [183]:
ismailasarr_finaldat = compile_dat('bfdb33aa', 'Ismaila-Sarr', 'Ismaila_Sarr', checkgames = True)
if isinstance(ismailasarr_finaldat, pd.DataFrame):
    ismailasarr_finaldat.to_csv(os.path.join(nest_folder, "ismailasarr_finaldat.csv"))

In [155]:
hughes_finaldat = compile_dat('a0666d3e', 'Will-Hughes', 'Will_Hughes', checkgames = True)
if isinstance(hughes_finaldat, pd.DataFrame):
    hughes_finaldat.to_csv(os.path.join(nest_folder, "hughes_finaldat.csv"))

In [157]:
schlupp_finaldat = compile_dat('3312f911', 'Jeffrey-Schlupp', 'Jeffrey_Schlupp', checkgames = True)
if isinstance(schlupp_finaldat, pd.DataFrame):
    schlupp_finaldat.to_csv(os.path.join(nest_folder, "schlupp_finaldat.csv"))

Crystal Palace Defenders

In [55]:
munoz_finaldat = compile_dat('778ef829', 'Daniel-Munoz', 'Daniel_Mu', checkgames = True)
if isinstance(munoz_finaldat, pd.DataFrame):
    munoz_finaldat.to_csv(os.path.join(nest_folder_def, "munoz_finaldat.csv"))

In [56]:
mitchell_finaldat = compile_dat('5cbd1eb0', 'Tyrick-Mitchell', 'Tyrick_Mitchell', checkgames = True)
if isinstance(mitchell_finaldat, pd.DataFrame):
    mitchell_finaldat.to_csv(os.path.join(nest_folder_def, "mitchell_finaldat.csv"))

In [57]:
crichards_finaldat = compile_dat('0a3d6d2b', 'Chris-Richards', 'Chris_Richards', checkgames = True)
if isinstance(crichards_finaldat, pd.DataFrame):
    crichards_finaldat.to_csv(os.path.join(nest_folder_def, "crichards_finaldat.csv"))

In [58]:
guehi_finaldat = compile_dat('d0706b27', 'Marc-Guehi', 'Marc_Gu', checkgames = True)
if isinstance(guehi_finaldat, pd.DataFrame):
    guehi_finaldat.to_csv(os.path.join(nest_folder_def, "guehi_finaldat.csv"))


In [59]:
clyne_finaldat = compile_dat('0442183b', 'Nathaniel-Clyne', 'Nathaniel_Clyne', checkgames = True)
if isinstance(clyne_finaldat, pd.DataFrame):
    clyne_finaldat.to_csv(os.path.join(nest_folder_def, "clyne_finaldat.csv"))

In [62]:
tchalobah_finaldat = compile_dat('5515376c', 'Trevoh-Chalobah', 'Trevoh_Chalobah', checkgames = True)
if isinstance(tchalobah_finaldat, pd.DataFrame):
    tchalobah_finaldat.to_csv(os.path.join(nest_folder_def, "tchalobah_finaldat.csv"))

In [63]:
holding_finaldat = compile_dat('d79a8c87', 'Rob-Holding', 'Rob_Holding', checkgames = True)
if isinstance(holding_finaldat, pd.DataFrame):
    holding_finaldat.to_csv(os.path.join(nest_folder_def, "holding_finaldat.csv"))

In [64]:
jward_finaldat = compile_dat('8a7ff278', 'Joel-Ward', 'Joel_Ward', checkgames = True)
if isinstance(jward_finaldat, pd.DataFrame):
    jward_finaldat.to_csv(os.path.join(nest_folder_def, "jward_finaldat.csv"))

## Everton

In [158]:
calvertlewin_finaldat = compile_dat('59e6e5bf', 'Dominic-Calvert-Lewin', 'Dominic_Calvert', checkgames = True)
if isinstance(calvertlewin_finaldat, pd.DataFrame):
    calvertlewin_finaldat.to_csv(os.path.join(nest_folder, "calvertlewin_finaldat.csv"))

In [159]:
beto_finaldat = compile_dat('ed5c0319', 'Beto', 'Norberto Bercique', checkgames = True)
if isinstance(beto_finaldat, pd.DataFrame):
    beto_finaldat.to_csv(os.path.join(nest_folder, "beto_finaldat.csv"))

In [160]:
broja_finaldat = compile_dat('97220da2', 'Armando-Broja', 'Armando_Broja', checkgames = True)
if isinstance(broja_finaldat, pd.DataFrame):
    broja_finaldat.to_csv(os.path.join(nest_folder, "broja_finaldat.csv"))

In [184]:
mcneil_finaldat = compile_dat('fc15fb84', 'Dwight-McNeil', 'Dwight_McNeil', checkgames = True)
if isinstance(mcneil_finaldat, pd.DataFrame):
    mcneil_finaldat.to_csv(os.path.join(nest_folder, "mcneil_finaldat.csv"))

In [163]:
gueye_finaldat = compile_dat('72c812f3', 'Idrissa-Gana-Gueye', 'Idrissa_Gueye', checkgames = True)
if isinstance(gueye_finaldat, pd.DataFrame):
    gueye_finaldat.to_csv(os.path.join(nest_folder, "gueye_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [164]:
harrison_finaldat = compile_dat('aa849a12', 'Jack-Harrison', 'Jack_Harrison', checkgames = True)
if isinstance(harrison_finaldat, pd.DataFrame):
    harrison_finaldat.to_csv(os.path.join(nest_folder, "harrison_finaldat.csv"))

In [165]:
doucoure_finaldat = compile_dat('02b29014', 'Abdoulaye-Doucoure', 'Abdoulaye_Douco', checkgames = True)
if isinstance(doucoure_finaldat, pd.DataFrame):
    doucoure_finaldat.to_csv(os.path.join(nest_folder, "doucoure_finaldat.csv"))

In [166]:
garner_finaldat = compile_dat('4e015693', 'James-Garner', 'James_Garner', checkgames = True)
if isinstance(garner_finaldat, pd.DataFrame):
    garner_finaldat.to_csv(os.path.join(nest_folder, "garner_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


Everton Defenders

In [65]:
mkeane_finaldat = compile_dat('c5b7c315', 'Michael-Keane', 'Michael_Keane', checkgames = True)
if isinstance(mkeane_finaldat, pd.DataFrame):
    mkeane_finaldat.to_csv(os.path.join(nest_folder_def, "mkeane_finaldat.csv"))

In [67]:
mykolenko_finaldat = compile_dat('30d4a2e5', 'Vitaliy-Mykolenko', 'Vitalii_Mykolenko', checkgames = False)
if isinstance(mykolenko_finaldat, pd.DataFrame):
    mykolenko_finaldat.to_csv(os.path.join(nest_folder_def, "mykolenko_finaldat.csv"))

In [68]:
coleman_finaldat = compile_dat('0420d84f', 'Seamus-Coleman', 'amus_Coleman', checkgames = True)
if isinstance(coleman_finaldat, pd.DataFrame):
    coleman_finaldat.to_csv(os.path.join(nest_folder_def, "coleman_finaldat.csv"))

In [69]:
tarkowski_finaldat = compile_dat('15ea812b', 'James-Tarkowski', 'James_Tarkowski', checkgames = True)
if isinstance(tarkowski_finaldat, pd.DataFrame):
    tarkowski_finaldat.to_csv(os.path.join(nest_folder_def, "tarkowski_finaldat.csv"))

In [70]:
branthwaite_finaldat = compile_dat('c1949191', 'Jarrad-Branthwaite', 'Jarrad_Branthwaite', checkgames = True)
if isinstance(branthwaite_finaldat, pd.DataFrame):
    branthwaite_finaldat.to_csv(os.path.join(nest_folder_def, "branthwaite_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [71]:
npatterson_finaldat = compile_dat('230f0471', 'Nathan-Patterson', 'Nathan_Patterson', checkgames = True)
if isinstance(npatterson_finaldat, pd.DataFrame):
    npatterson_finaldat.to_csv(os.path.join(nest_folder_def, "npatterson_finaldat.csv"))

In [72]:
ayoung_finaldat = compile_dat('be927d03', 'Ashley-Young', 'Ashley_Young', checkgames = True)
if isinstance(ayoung_finaldat, pd.DataFrame):
    ayoung_finaldat.to_csv(os.path.join(nest_folder_def, "ayoung_finaldat.csv"))

## Fulham

In [167]:
muniz_finaldat = compile_dat('a755db8c', 'Rodrigo-Muniz', 'Rodrigo_Muniz', checkgames = True)
if isinstance(muniz_finaldat, pd.DataFrame):
    muniz_finaldat.to_csv(os.path.join(nest_folder, "muniz_finaldat.csv"))

In [170]:
raul_earlydat = compile_dat('b561db50', 'Raul-Jimenez', 'Raúl_Jiménez', checkgames = False)
raul_latedat = compile_dat('b561db50', 'Raul-Jimenez', 'Jiménez', checkgames = False)
raul_finaldat = pd.concat([raul_latedat, raul_earlydat], join = "inner", ignore_index = True)
if isinstance(raul_finaldat, pd.DataFrame):
    raul_finaldat.to_csv(os.path.join(nest_folder, "raul_finaldat.csv"))

In [169]:
carlosvini_finaldat = compile_dat('8b529245', 'Carlos-Vinicius', 'Carlos Vin', checkgames = True)
if isinstance(carlosvini_finaldat, pd.DataFrame):
    carlosvini_finaldat.to_csv(os.path.join(nest_folder, "carlosvini_finaldat.csv"))

In [181]:
adama_finaldat = compile_dat('9a28eba4', 'Adama-Traore', 'Adama_Traor', checkgames = True)
if isinstance(adama_finaldat, pd.DataFrame):
    adama_finaldat.to_csv(os.path.join(nest_folder, "adama_finaldat.csv"))

In [172]:
smithrowe_finaldat = compile_dat('17695062', 'Emile-Smith-Rowe', 'Emile_Smith', checkgames = True)
if isinstance(smithrowe_finaldat, pd.DataFrame):
    smithrowe_finaldat.to_csv(os.path.join(nest_folder, "smithrowe_finaldat.csv"))

In [173]:
iwobi_finaldat = compile_dat('6ca5ec4b', 'Alex-Iwobi', 'Alex_Iwobi', checkgames = True)
if isinstance(iwobi_finaldat, pd.DataFrame):
    iwobi_finaldat.to_csv(os.path.join(nest_folder, "iwobi_finaldat.csv"))

In [174]:
andreas_earlydat = compile_dat('6639e500', 'Andreas-Pereira', 'Andreas_Pereira', checkgames = False)
andreas_latedat = compile_dat('6639e500', 'Andreas-Pereira', 'Hoelgebaum', checkgames = False)
andreas_finaldat = pd.concat([andreas_latedat, andreas_earlydat], join = "inner", ignore_index = True)
if isinstance(andreas_finaldat, pd.DataFrame):
    andreas_finaldat.to_csv(os.path.join(nest_folder, "andreas_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [175]:
lukic_finaldat = compile_dat('c6e8cf1f', 'Sasa-Lukic', 'Lukic', checkgames = True)
if isinstance(lukic_finaldat, pd.DataFrame):
    lukic_finaldat.to_csv(os.path.join(nest_folder, "lukic_finaldat.csv"))

In [176]:
reed_finaldat = compile_dat('803ae100', 'Harrison-Reed', 'Harrison_Reed', checkgames = True)
if isinstance(reed_finaldat, pd.DataFrame):
    reed_finaldat.to_csv(os.path.join(nest_folder, "reed_finaldat.csv"))

In [177]:
wilson_finaldat = compile_dat('c6dc9ecd', 'Harry-Wilson', 'Harry_Wilson', checkgames = True)
if isinstance(wilson_finaldat, pd.DataFrame):
    wilson_finaldat.to_csv(os.path.join(nest_folder, "wilson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [178]:
nelson_finaldat = compile_dat('c5bdb6e3', 'Reiss-Nelson', 'Reiss_Nelson', checkgames = True)
if isinstance(nelson_finaldat, pd.DataFrame):
    nelson_finaldat.to_csv(os.path.join(nest_folder, "nelson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [185]:
cairney_finaldat = compile_dat('a8748947', 'Tom-Cairney', 'Tom_Cairney', checkgames = True)
if isinstance(cairney_finaldat, pd.DataFrame):
    cairney_finaldat.to_csv(os.path.join(nest_folder, "cairney_finaldat.csv"))

In [186]:
berge_finaldat = compile_dat('d0b6129f', 'Sander-Berge', 'Sander_Berge', checkgames = True)
if isinstance(berge_finaldat, pd.DataFrame):
    berge_finaldat.to_csv(os.path.join(nest_folder, "berge_finaldat.csv"))

Fulham Defenders

In [73]:
arobinson_finaldat = compile_dat('289601e6', 'Antonee-Robinson', 'Antonee_Robinson', checkgames = True)
if isinstance(arobinson_finaldat, pd.DataFrame):
    arobinson_finaldat.to_csv(os.path.join(nest_folder_def, "arobinson_finaldat.csv"))

In [74]:
diop_finaldat = compile_dat('a712ca2b', 'Issa-Diop', 'Issa_Diop', checkgames = True)
if isinstance(diop_finaldat, pd.DataFrame):
    diop_finaldat.to_csv(os.path.join(nest_folder_def, "diop_finaldat.csv"))

In [75]:
tete_finaldat = compile_dat('77d7c96f', 'Kenny-Tete', 'Kenny_Tete', checkgames = True)
if isinstance(tete_finaldat, pd.DataFrame):
    tete_finaldat.to_csv(os.path.join(nest_folder_def, "tete_finaldat.csv"))

In [76]:
bassey_finaldat = compile_dat('a36524bf', 'Calvin-Bassey', 'Calvin_Bassey', checkgames = True)
if isinstance(bassey_finaldat, pd.DataFrame):
    bassey_finaldat.to_csv(os.path.join(nest_folder_def, "bassey_finaldat.csv"))

In [77]:
jandersen_finaldat = compile_dat('e34fc66d', 'Joachim-Andersen', 'Joachim_Andersen', checkgames = True)
if isinstance(jandersen_finaldat, pd.DataFrame):
    jandersen_finaldat.to_csv(os.path.join(nest_folder_def, "jandersen_finaldat.csv"))

In [78]:
castagne_finaldat = compile_dat('197640fd', 'Timothy-Castagne', 'Timothy_Castagne', checkgames = True)
if isinstance(castagne_finaldat, pd.DataFrame):
    castagne_finaldat.to_csv(os.path.join(nest_folder_def, "castagne_finaldat.csv"))

In [79]:
sessegnon_finaldat = compile_dat('6aa3e78b', 'Ryan-Sessegnon', 'Ryan_Sessegnon', checkgames = True)
if isinstance(sessegnon_finaldat, pd.DataFrame):
    sessegnon_finaldat.to_csv(os.path.join(nest_folder_def, "sessegnon_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


## Leicester

In [187]:
vardy_finaldat = compile_dat('45963054', 'Jamie-Vardy', 'Jamie_Vardy', checkgames = True)
if isinstance(vardy_finaldat, pd.DataFrame):
    vardy_finaldat.to_csv(os.path.join(nest_folder, "vardy_finaldat.csv"))

In [188]:
edouard_finaldat = compile_dat('0562b7f1', 'Odsonne-Edouard', 'Odsonne_', checkgames = True)
if isinstance(edouard_finaldat, pd.DataFrame):
    edouard_finaldat.to_csv(os.path.join(nest_folder, "edouard_finaldat.csv"))

In [189]:
daka_finaldat = compile_dat('ca45605e', 'Patson-Daka', 'Patson_Daka', checkgames = True)
if isinstance(daka_finaldat, pd.DataFrame):
    daka_finaldat.to_csv(os.path.join(nest_folder, "daka_finaldat.csv"))

In [190]:
buonanotte_finaldat = compile_dat('468a7a91', 'Facundo-Buonanotte', 'Facundo_B', checkgames = True)
if isinstance(buonanotte_finaldat, pd.DataFrame):
    buonanotte_finaldat.to_csv(os.path.join(nest_folder, "buonanotte_finaldat.csv"))

In [194]:
ndidi_finaldat = compile_dat('6b47c5db', 'Wilfred-Ndidi', 'Wilfred_Ndidi', checkgames = True)
if isinstance(ndidi_finaldat, pd.DataFrame):
    ndidi_finaldat.to_csv(os.path.join(nest_folder, "ndidi_finaldat.csv"))

In [195]:
decordovareid_earlydat = compile_dat('0f7533cd', 'Bobby-Reid', 'Bobby_Reid', checkgames = False)
decordovareid_latedat = compile_dat('0f7533cd', 'Bobby-Reid', 'Bobby_De', checkgames = False)
decordovareid_finaldat = pd.concat([decordovareid_latedat, decordovareid_earlydat], join = "inner", ignore_index = True)
if isinstance(decordovareid_finaldat, pd.DataFrame):
    decordovareid_finaldat.to_csv(os.path.join(nest_folder, "decordovareid_finaldat.csv"))

In [196]:
winks_finaldat = compile_dat('2f7acede', 'Harry-Winks', 'Harry_Winks', checkgames = True)
if isinstance(winks_finaldat, pd.DataFrame):
    winks_finaldat.to_csv(os.path.join(nest_folder, "winks_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [197]:
jayew_finaldat = compile_dat('da052c14', 'Jordan-Ayew', 'Jordan_Ayew', checkgames = True)
if isinstance(jayew_finaldat, pd.DataFrame):
    jayew_finaldat.to_csv(os.path.join(nest_folder, "jayew_finaldat.csv"))

In [198]:
skipp_finaldat = compile_dat('6250083a', 'Oliver-Skipp', 'Oliver_Skipp', checkgames = True)
if isinstance(skipp_finaldat, pd.DataFrame):
    skipp_finaldat.to_csv(os.path.join(nest_folder, "skipp_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [199]:
soumare_finaldat = compile_dat('ae59b359', 'Boubakary-Soumare', 'Boubakary_Sou', checkgames = True)
if isinstance(soumare_finaldat, pd.DataFrame):
    soumare_finaldat.to_csv(os.path.join(nest_folder, "soumare_finaldat.csv"))

In [200]:
choudhury_finaldat = compile_dat('7d2d3329', 'Hamza-Choudhury', 'Hamza_Choudhury', checkgames = True)
if isinstance(choudhury_finaldat, pd.DataFrame):
    choudhury_finaldat.to_csv(os.path.join(nest_folder, "choudhury_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


Leicester Defenders

In [80]:
jjustin_finaldat = compile_dat('fb614c44', 'James-Justin', 'James_Justin', checkgames = True)
if isinstance(jjustin_finaldat, pd.DataFrame):
    jjustin_finaldat.to_csv(os.path.join(nest_folder_def, "jjustin_finaldat.csv"))

In [81]:
kristiansen_finaldat = compile_dat('505486d6', 'Victor-Bernth-Kristiansen', 'Victor_Kristiansen', checkgames = True)
if isinstance(kristiansen_finaldat, pd.DataFrame):
    kristiansen_finaldat.to_csv(os.path.join(nest_folder_def, "kristiansen_finaldat.csv"))

In [82]:
vestergaard_finaldat = compile_dat('1900032e', 'Jannik-Vestergaard', 'Jannik_Vestergaard', checkgames = True)
if isinstance(vestergaard_finaldat, pd.DataFrame):
    vestergaard_finaldat.to_csv(os.path.join(nest_folder_def, "vestergaard_finaldat.csv"))

In [83]:
coady_finaldat = compile_dat('2928dca2', 'Conor-Coady', 'Conor_Coady', checkgames = True)
if isinstance(coady_finaldat, pd.DataFrame):
    coady_finaldat.to_csv(os.path.join(nest_folder_def, "coady_finaldat.csv"))

In [84]:
ricardop_finaldat = compile_dat('75a72a99', 'Ricardo-Pereira', 'Barbosa Pereira', checkgames = True)
if isinstance(ricardop_finaldat, pd.DataFrame):
    ricardop_finaldat.to_csv(os.path.join(nest_folder_def, "ricardop_finaldat.csv"))

In [85]:
lthomas_finaldat = compile_dat('fc027d02', 'Luke-Thomas', 'Luke_Thomas', checkgames = True)
if isinstance(lthomas_finaldat, pd.DataFrame):
    lthomas_finaldat.to_csv(os.path.join(nest_folder_def, "lthomas_finaldat.csv"))

## Liverpool

In [201]:
szoboszlai_finaldat = compile_dat('934e1968', 'Dominik-Szoboszlai', 'Dominik_Szoboszlai', checkgames = True)
if isinstance(szoboszlai_finaldat, pd.DataFrame):
    szoboszlai_finaldat.to_csv(os.path.join(nest_folder, "szoboszlai_finaldat.csv"))

In [202]:
macallister_finaldat = compile_dat('83d074ff', 'Alexis-Mac-Allister', 'Alexis_Mac', checkgames = True)
if isinstance(macallister_finaldat, pd.DataFrame):
    macallister_finaldat.to_csv(os.path.join(nest_folder, "macallister_finaldat.csv"))

In [203]:
gravenberch_finaldat = compile_dat('b8e740fb', 'Ryan-Gravenberch', 'Ryan_Gravenberch', checkgames = True)
if isinstance(gravenberch_finaldat, pd.DataFrame):
    gravenberch_finaldat.to_csv(os.path.join(nest_folder, "gravenberch_finaldat.csv"))

In [204]:
elliott_finaldat = compile_dat('b9e1436c', 'Harvey-Elliott', 'Harvey_Elliott', checkgames = True)
if isinstance(elliott_finaldat, pd.DataFrame):
    elliott_finaldat.to_csv(os.path.join(nest_folder, "elliott_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [205]:
curtis_finaldat = compile_dat('4fb9c88f', 'Curtis-Jones', 'Curtis_Jones', checkgames = True)
if isinstance(curtis_finaldat, pd.DataFrame):
    curtis_finaldat.to_csv(os.path.join(nest_folder, "curtis_finaldat.csv"))

In [206]:
endo_finaldat = compile_dat('c149016b', 'Wataru-Endo', 'Wataru_Endo', checkgames = True)
if isinstance(endo_finaldat, pd.DataFrame):
    endo_finaldat.to_csv(os.path.join(nest_folder, "endo_finaldat.csv"))

Liverpool Defenders

In [88]:
taa_finaldat = compile_dat('cd1acf9d', 'Trent-Alexander-Arnold', 'Trent_Alexander', checkgames = True)
if isinstance(taa_finaldat, pd.DataFrame):
    taa_finaldat.to_csv(os.path.join(nest_folder_def, "taa_finaldat.csv"))

In [89]:
robertson_finaldat = compile_dat('2e4f5f03', 'Andrew-Robertson', 'Andrew_Robertson', checkgames = True)
if isinstance(robertson_finaldat, pd.DataFrame):
    robertson_finaldat.to_csv(os.path.join(nest_folder_def, "robertson_finaldat.csv"))

In [90]:
virgil_finaldat = compile_dat('e06683ca', 'Virgil-van-Dijk', 'Virgil_van', checkgames = True)
if isinstance(virgil_finaldat, pd.DataFrame):
    virgil_finaldat.to_csv(os.path.join(nest_folder_def, "virgil_finaldat.csv"))

In [91]:
konate_finaldat = compile_dat('5ed9b537', 'Ibrahima-Konate', 'Ibrahima_Kona', checkgames = True)
if isinstance(konate_finaldat, pd.DataFrame):
    konate_finaldat.to_csv(os.path.join(nest_folder_def, "konate_finaldat.csv"))

In [92]:
cbradley_finaldat = compile_dat('bbd67769', 'Conor-Bradley', 'Conor_Bradley', checkgames = True)
if isinstance(cbradley_finaldat, pd.DataFrame):
    cbradley_finaldat.to_csv(os.path.join(nest_folder_def, "cbradley_finaldat.csv"))

In [93]:
tsimikas_finaldat = compile_dat('f315ca93', 'Kostas-Tsimikas', 'Tsimikas', checkgames = True)
if isinstance(tsimikas_finaldat, pd.DataFrame):
    tsimikas_finaldat.to_csv(os.path.join(nest_folder_def, "tsimikas_finaldat.csv"))

In [94]:
quansah_finaldat = compile_dat('4125cb98', 'Jarell-Quansah', 'Jarell_Quansah', checkgames = True)
if isinstance(quansah_finaldat, pd.DataFrame):
    quansah_finaldat.to_csv(os.path.join(nest_folder_def, "quansah_finaldat.csv"))

In [95]:
gomez_finaldat = compile_dat('7a11550b', 'Joe-Gomez', 'Joseph_Gomez', checkgames = True)
if isinstance(gomez_finaldat, pd.DataFrame):
    gomez_finaldat.to_csv(os.path.join(nest_folder_def, "gomez_finaldat.csv"))

## Manchester City

In [207]:
haaland_finaldat = compile_dat('1f44ac21', 'Erling-Haaland', 'Erling_Haaland', checkgames = True)
if isinstance(haaland_finaldat, pd.DataFrame):
    haaland_finaldat.to_csv(os.path.join(nest_folder, "haaland_finaldat.csv"))

In [208]:
debruyne_finaldat = compile_dat('e46012d4', 'Kevin-De-Bruyne', 'Kevin_De Bruyne', checkgames = True)
if isinstance(debruyne_finaldat, pd.DataFrame):
    debruyne_finaldat.to_csv(os.path.join(nest_folder, "debruyne_finaldat.csv"))

In [209]:
bernardo_finaldat = compile_dat('3eb22ec9', 'Bernardo-Silva', 'Veiga de Carvalho', checkgames = True)
if isinstance(bernardo_finaldat, pd.DataFrame):
    bernardo_finaldat.to_csv(os.path.join(nest_folder, "bernardo_finaldat.csv"))

In [210]:
kovacic_finaldat = compile_dat('79c0821a', 'Mateo-Kovacic', 'Mateo_Kova', checkgames = True)
if isinstance(kovacic_finaldat, pd.DataFrame):
    kovacic_finaldat.to_csv(os.path.join(nest_folder, "kovacic_finaldat.csv"))

In [212]:
doku_finaldat = compile_dat('fffea3e5', 'Jeremy-Doku', 'Doku', checkgames = True)
if isinstance(doku_finaldat, pd.DataFrame):
    doku_finaldat.to_csv(os.path.join(nest_folder, "doku_finaldat.csv"))

In [247]:
matheusn_finaldat = compile_dat('e6af02e0', 'Matheus-Nunes', 'Matheus Luiz_Nunes', checkgames = False)
if isinstance(matheusn_finaldat, pd.DataFrame):
    matheusn_finaldat.to_csv(os.path.join(nest_folder, "matheusn_finaldat.csv"))

#checkgames false because there is an extra row, for a game that Matheus Nunes technically did not play against Leeds. However, that row exists 
#because he managed to pickup a red card despite not playing.

In [220]:
grealish_finaldat = compile_dat('b0b4fd3e', 'Jack-Grealish', 'Jack_Grealish', checkgames = True)
if isinstance(grealish_finaldat, pd.DataFrame):
    grealish_finaldat.to_csv(os.path.join(nest_folder, "grealish_finaldat.csv"))

In [221]:
gundogan_finaldat = compile_dat('819b3158', 'Ilkay-Gundogan', 'Ilkay_G', checkgames = True)
if isinstance(gundogan_finaldat, pd.DataFrame):
    gundogan_finaldat.to_csv(os.path.join(nest_folder, "gundogan_finaldat.csv"))

In [248]:
foden_finaldat = compile_dat('ed1e53f3', 'Phil-Foden', 'Phil_Foden', checkgames = True)
if isinstance(foden_finaldat, pd.DataFrame):
    foden_finaldat.to_csv(os.path.join(nest_folder, "foden_finaldat.csv"))

In [223]:
bobb_finaldat = compile_dat('eed2427e', 'Oscar-Bobb', 'Oscar_Bobb', checkgames = True)
if isinstance(bobb_finaldat, pd.DataFrame):
    bobb_finaldat.to_csv(os.path.join(nest_folder, "bobb_finaldat.csv"))

In [224]:
rodri_finaldat = compile_dat('6434f10d', 'Rodri', 'Rodrigo_Hernandez', checkgames = True)
if isinstance(rodri_finaldat, pd.DataFrame):
    rodri_finaldat.to_csv(os.path.join(nest_folder, "rodri_finaldat.csv"))

Manchester City Defenders

In [96]:
rlewis_finaldat = compile_dat('b57e066e', 'Rico-Lewis', 'Rico_Lewis', checkgames = True)
if isinstance(rlewis_finaldat, pd.DataFrame):
    rlewis_finaldat.to_csv(os.path.join(nest_folder_def, "rlewis_finaldat.csv"))

In [97]:
gvardiol_finaldat = compile_dat('5ad50391', 'Josko-Gvardiol', 'Gvardiol', checkgames = True)
if isinstance(gvardiol_finaldat, pd.DataFrame):
    gvardiol_finaldat.to_csv(os.path.join(nest_folder_def, "gvardiol_finaldat.csv"))

In [99]:
akanji_finaldat = compile_dat('89ac64a6', 'Manuel-Akanji', 'Manuel_Akanji', checkgames = True)
if isinstance(akanji_finaldat, pd.DataFrame):
    akanji_finaldat.to_csv(os.path.join(nest_folder_def, "akanji_finaldat.csv"))

In [101]:
ruben_finaldat = compile_dat('31c69ef1', 'Ruben-Dias', 'Gato Alves Dias', checkgames = True)
if isinstance(ruben_finaldat, pd.DataFrame):
    ruben_finaldat.to_csv(os.path.join(nest_folder_def, "ruben_finaldat.csv"))

In [102]:
ake_finaldat = compile_dat('eaeca114', 'Nathan-Ake', 'Nathan_Ak', checkgames = True)
if isinstance(ake_finaldat, pd.DataFrame):
    ake_finaldat.to_csv(os.path.join(nest_folder_def, "ake_finaldat.csv"))

In [103]:
stones_finaldat = compile_dat('5eecec3d', 'John-Stones', 'John_Stones', checkgames = True)
if isinstance(stones_finaldat, pd.DataFrame):
    stones_finaldat.to_csv(os.path.join(nest_folder_def, "stones_finaldat.csv"))

In [111]:
kwalker_finaldat = compile_dat('86dd77d1', 'Kyle-Walker', 'Kyle_Walker_', checkgames = True)
if isinstance(kwalker_finaldat, pd.DataFrame):
    kwalker_finaldat.to_csv(os.path.join(nest_folder_def, "kwalker_finaldat.csv"))

## Manchester United

In [225]:
hojlund_finaldat = compile_dat('491a433d', 'Rasmus-Hojlund', 'Rasmus_H', checkgames = True)
if isinstance(hojlund_finaldat, pd.DataFrame):
    hojlund_finaldat.to_csv(os.path.join(nest_folder, "hojlund_finaldat.csv"))

In [226]:
brunof_finaldat = compile_dat('507c7bdf', 'Bruno-Fernandes', 'Borges Fernandes', checkgames = True)
if isinstance(brunof_finaldat, pd.DataFrame):
    brunof_finaldat.to_csv(os.path.join(nest_folder, "brunof_finaldat.csv"))

In [227]:
garnacho_finaldat = compile_dat('7aa8adfe', 'Alejandro-Garnacho', 'Alejandro_Garnacho', checkgames = True)
if isinstance(garnacho_finaldat, pd.DataFrame):
    garnacho_finaldat.to_csv(os.path.join(nest_folder, "garnacho_finaldat.csv"))

In [228]:
rashford_finaldat = compile_dat('a1d5bd30', 'Marcus-Rashford', 'Marcus_Rashford', checkgames = True)
if isinstance(rashford_finaldat, pd.DataFrame):
    rashford_finaldat.to_csv(os.path.join(nest_folder, "rashford_finaldat.csv"))

In [229]:
casemiro_finaldat = compile_dat('4d224fe8', 'Casemiro', 'Casimiro', checkgames = True)
if isinstance(casemiro_finaldat, pd.DataFrame):
    casemiro_finaldat.to_csv(os.path.join(nest_folder, "casemiro_finaldat.csv"))

In [230]:
mainoo_finaldat = compile_dat('c6220452', 'Kobbie-Mainoo', 'Kobbie_Mainoo', checkgames = True)
if isinstance(mainoo_finaldat, pd.DataFrame):
    mainoo_finaldat.to_csv(os.path.join(nest_folder, "mainoo_finaldat.csv"))

In [231]:
mount_finaldat = compile_dat('9674002f', 'Mason-Mount', 'Mason_Mount', checkgames = True)
if isinstance(mount_finaldat, pd.DataFrame):
    mount_finaldat.to_csv(os.path.join(nest_folder, "mount_finaldat.csv"))

In [232]:
antony_finaldat = compile_dat('99127249', 'Antony', 'Antony', checkgames = True)
if isinstance(antony_finaldat, pd.DataFrame):
    antony_finaldat.to_csv(os.path.join(nest_folder, "antony_finaldat.csv"))

In [233]:
eriksen_finaldat = compile_dat('980522ec', 'Christian-Eriksen', 'Christian_Eriksen', checkgames = True)
if isinstance(eriksen_finaldat, pd.DataFrame):
    eriksen_finaldat.to_csv(os.path.join(nest_folder, "eriksen_finaldat.csv"))

Manchester United Defenders

In [105]:
dalot_finaldat = compile_dat('d9565625', 'Diogo-Dalot', 'Diogo_Dalot', checkgames = True)
if isinstance(dalot_finaldat, pd.DataFrame):
    dalot_finaldat.to_csv(os.path.join(nest_folder_def, "dalot_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [106]:
hmaguire_finaldat = compile_dat('d8931174', 'Harry-Maguire', 'Harry_Maguire', checkgames = True)
if isinstance(hmaguire_finaldat, pd.DataFrame):
    hmaguire_finaldat.to_csv(os.path.join(nest_folder_def, "hmaguire_finaldat.csv"))

In [107]:
lisandro_finaldat = compile_dat('bac46a10', 'Lisandro-Martinez', 'Lisandro_Mart', checkgames = True)
if isinstance(lisandro_finaldat, pd.DataFrame):
    lisandro_finaldat.to_csv(os.path.join(nest_folder_def, "lisandro_finaldat.csv"))

In [108]:
jevans_finaldat = compile_dat('f8fcd2a5', 'Jonny-Evans', 'Jonny_Evans', checkgames = True)
if isinstance(jevans_finaldat, pd.DataFrame):
    jevans_finaldat.to_csv(os.path.join(nest_folder_def, "jevans_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


The number of Premier League games played by Jonny-Evans(172) does not match the number of rows in the dataframe (144)


In [109]:
lindelof_finaldat = compile_dat('f5deef4c', 'Victor-Lindelof', 'Victor_Lindel', checkgames = True)
if isinstance(lindelof_finaldat, pd.DataFrame):
    lindelof_finaldat.to_csv(os.path.join(nest_folder_def, "lindelof_finaldat.csv"))

In [110]:
shaw_finaldat = compile_dat('9c94165b', 'Luke-Shaw', 'Luke_Shaw', checkgames = True)
if isinstance(shaw_finaldat, pd.DataFrame):
    shaw_finaldat.to_csv(os.path.join(nest_folder_def, "shaw_finaldat.csv"))

## Newcastle

In [234]:
isak_finaldat = compile_dat('8e92be30', 'Alexander-Isak', 'Alexander_Isak', checkgames = True)
if isinstance(isak_finaldat, pd.DataFrame):
    isak_finaldat.to_csv(os.path.join(nest_folder, "isak_finaldat.csv"))

In [235]:
cwilson_finaldat = compile_dat('c596fcb0', 'Callum-Wilson', 'Callum_Wilson', checkgames = True)
if isinstance(cwilson_finaldat, pd.DataFrame):
    cwilson_finaldat.to_csv(os.path.join(nest_folder, "cwilson_finaldat.csv"))

In [236]:
hbarnes_finaldat = compile_dat('3ea50f67', 'Harvey-Barnes', 'Harvey_Barnes', checkgames = True)
if isinstance(hbarnes_finaldat, pd.DataFrame):
    hbarnes_finaldat.to_csv(os.path.join(nest_folder, "hbarnes_finaldat.csv"))

In [237]:
gordon_finaldat = compile_dat('2bd83368', 'Anthony-Gordon', 'Anthony_Gordon', checkgames = True)
if isinstance(gordon_finaldat, pd.DataFrame):
    gordon_finaldat.to_csv(os.path.join(nest_folder, "gordon_finaldat.csv"))

In [238]:
joelinton_finaldat = compile_dat('c17bfb65', 'Joelinton', 'Joelinton', checkgames = True)
if isinstance(joelinton_finaldat, pd.DataFrame):
    joelinton_finaldat.to_csv(os.path.join(nest_folder, "joelinton_finaldat.csv"))

In [239]:
brunog_finaldat = compile_dat('82518f62', 'Bruno-Guimaraes', 'Bruno_Gui', checkgames = True)
if isinstance(brunog_finaldat, pd.DataFrame):
    brunog_finaldat.to_csv(os.path.join(nest_folder, "brunog_finaldat.csv"))

In [240]:
jmurphy_finaldat = compile_dat('de112b84', 'Jacob-Murphy', 'Jacob_Murphy', checkgames = True)
if isinstance(jmurphy_finaldat, pd.DataFrame):
    jmurphy_finaldat.to_csv(os.path.join(nest_folder, "jmurphy_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [241]:
slongstaff_finaldat = compile_dat('a2b105e0', 'Sean-Longstaff', 'Sean_Longstaff', checkgames = True)
if isinstance(slongstaff_finaldat, pd.DataFrame):
    slongstaff_finaldat.to_csv(os.path.join(nest_folder, "slongstaff_finaldat.csv"))

In [242]:
almiron_finaldat = compile_dat('862a1c15', 'Miguel-Almiron', 'Miguel_Almir', checkgames = True)
if isinstance(almiron_finaldat, pd.DataFrame):
    almiron_finaldat.to_csv(os.path.join(nest_folder, "almiron_finaldat.csv"))

In [243]:
willock_finaldat = compile_dat('a3b03921', 'Joe-Willock', 'Willock', checkgames = True)
if isinstance(willock_finaldat, pd.DataFrame):
    willock_finaldat.to_csv(os.path.join(nest_folder, "willock_finaldat.csv"))

In [245]:
hayden_finaldat = compile_dat('c951a6df', 'Isaac-Hayden', 'Isaac_Hayden', checkgames = True)
if isinstance(hayden_finaldat, pd.DataFrame):
    hayden_finaldat.to_csv(os.path.join(nest_folder, "hayden_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [246]:
miley_finaldat = compile_dat('2c6835e5', 'Lewis-Miley', 'Lewis_Miley', checkgames = True)
if isinstance(miley_finaldat, pd.DataFrame):
    miley_finaldat.to_csv(os.path.join(nest_folder, "miley_finaldat.csv"))

Newcastle Defenders

In [112]:
krafth_finaldat = compile_dat('77cf6852', 'Emil-Krafth', 'Emil_Krafth', checkgames = True)
if isinstance(krafth_finaldat, pd.DataFrame):
    krafth_finaldat.to_csv(os.path.join(nest_folder_def, "krafth_finaldat.csv"))

In [113]:
livramento_finaldat = compile_dat('afed6722', 'Valentino-Livramento', 'Livramento', checkgames = True)
if isinstance(livramento_finaldat, pd.DataFrame):
    livramento_finaldat.to_csv(os.path.join(nest_folder_def, "livramento_finaldat.csv"))

In [114]:
lhall_finaldat = compile_dat('da011f18', 'Lewis-Hall', 'Lewis_Hall', checkgames = True)
if isinstance(lhall_finaldat, pd.DataFrame):
    lhall_finaldat.to_csv(os.path.join(nest_folder_def, "lhall_finaldat.csv"))

In [115]:
lkelly_finaldat = compile_dat('f31def1e', 'Lloyd-Kelly', 'Lloyd_Kelly', checkgames = True)
if isinstance(lkelly_finaldat, pd.DataFrame):
    lkelly_finaldat.to_csv(os.path.join(nest_folder_def, "lkelly_finaldat.csv"))

In [116]:
burn_finaldat = compile_dat('b2d31e83', 'Dan-Burn', 'Dan_Burn', checkgames = True)
if isinstance(burn_finaldat, pd.DataFrame):
    burn_finaldat.to_csv(os.path.join(nest_folder_def, "burn_finaldat.csv"))

In [117]:
trippier_finaldat = compile_dat('21512407', 'Kieran-Trippier', 'Kieran_Trippier', checkgames = True)
if isinstance(trippier_finaldat, pd.DataFrame):
    trippier_finaldat.to_csv(os.path.join(nest_folder_def, "trippier_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [118]:
botman_finaldat = compile_dat('ccce7025', 'Sven-Botman', 'Sven_Botman', checkgames = True)
if isinstance(botman_finaldat, pd.DataFrame):
    botman_finaldat.to_csv(os.path.join(nest_folder_def, "botman_finaldat.csv"))

In [145]:
lascelles_finaldat = compile_dat('450ab6fc', 'Jamaal-Lascelles', 'Jamaal_Lascelles', checkgames = False)
if isinstance(lascelles_finaldat, pd.DataFrame):
    lascelles_finaldat.to_csv(os.path.join(nest_folder_def, "lascelles_finaldat.csv"))
    
#checkgames false because of the matheus nunes problem 

In [120]:
targett_finaldat = compile_dat('e514ab62', 'Matt-Targett', 'Matt_Targett', checkgames = True)
if isinstance(targett_finaldat, pd.DataFrame):
    targett_finaldat.to_csv(os.path.join(nest_folder_def, "targett_finaldat.csv"))

In [121]:
schar_finaldat = compile_dat('c8aa95da', 'Fabian-Schar', 'Fabian_Sch', checkgames = True)
if isinstance(schar_finaldat, pd.DataFrame):
    schar_finaldat.to_csv(os.path.join(nest_folder_def, "schar_finaldat.csv"))

## Nottingham Forest

In [249]:
wood_finaldat = compile_dat('4e9a0555', 'Chris-Wood', 'Chris_Wood', checkgames = True)
if isinstance(wood_finaldat, pd.DataFrame):
    wood_finaldat.to_csv(os.path.join(nest_folder, "wood_finaldat.csv"))

In [250]:
awoniyi_finaldat = compile_dat('e5478b87', 'Taiwo-Awoniyi', 'Taiwo_Awoniyi', checkgames = True)
if isinstance(awoniyi_finaldat, pd.DataFrame):
    awoniyi_finaldat.to_csv(os.path.join(nest_folder, "awoniyi_finaldat.csv"))

In [251]:
dennis_finaldat = compile_dat('2c44a35d', 'Emmanuel-Dennis', 'Emmanuel_Dennis', checkgames = True)
if isinstance(dennis_finaldat, pd.DataFrame):
    dennis_finaldat.to_csv(os.path.join(nest_folder, "dennis_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [252]:
gibbswhite_finaldat = compile_dat('32f60ed8', 'Morgan-Gibbs-White', 'Morgan_Gibbs', checkgames = True)
if isinstance(gibbswhite_finaldat, pd.DataFrame):
    gibbswhite_finaldat.to_csv(os.path.join(nest_folder, "gibbswhite_finaldat.csv"))

In [253]:
anderson_finaldat = compile_dat('de31038e', 'Elliot-Anderson', 'Elliot_Anderson', checkgames = True)
if isinstance(anderson_finaldat, pd.DataFrame):
    anderson_finaldat.to_csv(os.path.join(nest_folder, "anderson_finaldat.csv"))

In [254]:
yates_finaldat = compile_dat('c1fe2a48', 'Ryan-Yates', 'Ryan_Yates', checkgames = True)
if isinstance(yates_finaldat, pd.DataFrame):
    yates_finaldat.to_csv(os.path.join(nest_folder, "yates_finaldat.csv"))

In [255]:
hudsonodoi_finaldat = compile_dat('15f3ec41', 'Callum-Hudson-Odoi', 'Callum_Hudson', checkgames = True)
if isinstance(hudsonodoi_finaldat, pd.DataFrame):
    hudsonodoi_finaldat.to_csv(os.path.join(nest_folder, "hudsonodoi_finaldat.csv"))

In [258]:
elanga_finaldat = compile_dat('2fba6108', 'Anthony-Elanga', 'Anthony_Elanga', checkgames = True)
if isinstance(elanga_finaldat, pd.DataFrame):
    elanga_finaldat.to_csv(os.path.join(nest_folder, "elanga_finaldat.csv"))

In [259]:
sangare_finaldat = compile_dat('bced0375', 'Ibrahim-Sangare', 'Ibrahim_Sangar', checkgames = True)
if isinstance(sangare_finaldat, pd.DataFrame):
    sangare_finaldat.to_csv(os.path.join(nest_folder, "sangare_finaldat.csv"))

In [260]:
danilo_finaldat = compile_dat('a816dbfb', 'Danilo', 'Danilo_dos Santos', checkgames = True)
if isinstance(danilo_finaldat, pd.DataFrame):
    danilo_finaldat.to_csv(os.path.join(nest_folder, "danilo_finaldat.csv"))

In [261]:
dominguez_finaldat = compile_dat('20b3a502', 'Nicolas-Dominguez', 'Domínguez', checkgames = True)
if isinstance(dominguez_finaldat, pd.DataFrame):
    dominguez_finaldat.to_csv(os.path.join(nest_folder, "dominguez_finaldat.csv"))

In [262]:
wardprowse_finaldat = compile_dat('3515d404', 'James-Ward-Prowse', 'James_Ward', checkgames = True)
if isinstance(wardprowse_finaldat, pd.DataFrame):
    wardprowse_finaldat.to_csv(os.path.join(nest_folder, "wardprowse_finaldat.csv"))

Nottingham Forest Defenders

In [144]:
aina_finaldat = compile_dat('246d153b', 'Ola-Aina', 'Aina', checkgames = True)
if isinstance(aina_finaldat, pd.DataFrame):
    aina_finaldat.to_csv(os.path.join(nest_folder_def, "aina_finaldat.csv"))

In [123]:
murillo_finaldat = compile_dat('1704b0b8', 'Murillo', 'Murillo', checkgames = True)
if isinstance(murillo_finaldat, pd.DataFrame):
    murillo_finaldat.to_csv(os.path.join(nest_folder_def, "murillo_finaldat.csv"))

In [124]:
nwilliams_finaldat = compile_dat('dd323728', 'Neco-Williams', 'Neco_Williams', checkgames = True)
if isinstance(nwilliams_finaldat, pd.DataFrame):
    nwilliams_finaldat.to_csv(os.path.join(nest_folder_def, "nwilliams_finaldat.csv"))

In [125]:
boly_finaldat = compile_dat('a4ac4b8f', 'Willy-Boly', 'Willy_Boly', checkgames = True)
if isinstance(boly_finaldat, pd.DataFrame):
    boly_finaldat.to_csv(os.path.join(nest_folder_def, "boly_finaldat.csv"))

In [126]:
toffolo_finaldat = compile_dat('a105d46a', 'Harry-Toffolo', 'Harry_Toffolo', checkgames = True)
if isinstance(toffolo_finaldat, pd.DataFrame):
    toffolo_finaldat.to_csv(os.path.join(nest_folder_def, "toffolo_finaldat.csv"))

In [127]:
amoreno_finaldat = compile_dat('16e9d0ea', 'Alex-Moreno', 'Alexandre_Moreno', checkgames = True)
if isinstance(amoreno_finaldat, pd.DataFrame):
    amoreno_finaldat.to_csv(os.path.join(nest_folder_def, "amoreno_finaldat.csv"))

In [128]:
omobamidele_finaldat = compile_dat('c393a6c4', 'Andrew-Omobamidele', 'Andrew_Omobamidele', checkgames = True)
if isinstance(omobamidele_finaldat, pd.DataFrame):
    omobamidele_finaldat.to_csv(os.path.join(nest_folder_def, "omobamidele_finaldat.csv"))

## Southampton

In [263]:
aarmstrong_finaldat = compile_dat('68c720b5', 'Adam-Armstrong', 'Adam_Armstrong', checkgames = True)
if isinstance(aarmstrong_finaldat, pd.DataFrame):
    aarmstrong_finaldat.to_csv(os.path.join(nest_folder, "aarmstrong_finaldat.csv"))

In [264]:
archer_finaldat = compile_dat('05e8ca6d', 'Cameron-Archer', 'Cameron_Archer', checkgames = True)
if isinstance(archer_finaldat, pd.DataFrame):
    archer_finaldat.to_csv(os.path.join(nest_folder, "archer_finaldat.csv"))

In [267]:
aribo_finaldat = compile_dat('328f7d51', 'Joe-Aribo', 'Aribo', checkgames = True)
if isinstance(aribo_finaldat, pd.DataFrame):
    aribo_finaldat.to_csv(os.path.join(nest_folder, "aribo_finaldat.csv"))

In [268]:
smallbone_finaldat = compile_dat('5e105217', 'William-Smallbone', 'William_Smallbone', checkgames = True)
if isinstance(smallbone_finaldat, pd.DataFrame):
    smallbone_finaldat.to_csv(os.path.join(nest_folder, "smallbone_finaldat.csv"))

In [269]:
downes_finaldat = compile_dat('69fdb896', 'Flynn-Downes', 'Flynn_Downes', checkgames = True)
if isinstance(downes_finaldat, pd.DataFrame):
    downes_finaldat.to_csv(os.path.join(nest_folder, "downes_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [270]:
brereton_finaldat = compile_dat('57827369', 'Ben-Brereton', 'Ben_Brereton', checkgames = True)
if isinstance(brereton_finaldat, pd.DataFrame):
    brereton_finaldat.to_csv(os.path.join(nest_folder, "brereton_finaldat.csv"))

In [271]:
lallana_finaldat = compile_dat('99813635', 'Adam-Lallana', 'Adam_Lallana', checkgames = True)
if isinstance(lallana_finaldat, pd.DataFrame):
    lallana_finaldat.to_csv(os.path.join(nest_folder, "lallana_finaldat.csv"))

In [272]:
rfraser_finaldat = compile_dat('d56543a0', 'Ryan-Fraser', 'Ryan_Fraser', checkgames = True)
if isinstance(rfraser_finaldat, pd.DataFrame):
    rfraser_finaldat.to_csv(os.path.join(nest_folder, "rfraser_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [273]:
sulemana_finaldat = compile_dat('a62f8bf1', 'Kamaldeen-Sulemana', 'Kamaldeen_Sulemana', checkgames = True)
if isinstance(sulemana_finaldat, pd.DataFrame):
    sulemana_finaldat.to_csv(os.path.join(nest_folder, "sulemana_finaldat.csv"))

Southampton Defenders

In [129]:
walkerpeters_finaldat = compile_dat('984a5a64', 'Kyle-Walker-Peters', 'Walker-Peters', checkgames = True)
if isinstance(walkerpeters_finaldat, pd.DataFrame):
    walkerpeters_finaldat.to_csv(os.path.join(nest_folder_def, "walkerpeters_finaldat.csv"))


In [130]:
bednarek_finaldat = compile_dat('4115ce4e', 'Jan-Bednarek', 'Jan_Bednarek', checkgames = True)
if isinstance(bednarek_finaldat, pd.DataFrame):
    bednarek_finaldat.to_csv(os.path.join(nest_folder_def, "bednarek_finaldat.csv"))

In [131]:
jstephens_finaldat = compile_dat('6adbc307', 'Jack-Stephens', 'Jack_Stephens', checkgames = True)
if isinstance(jstephens_finaldat, pd.DataFrame):
    jstephens_finaldat.to_csv(os.path.join(nest_folder_def, "jstephens_finaldat.csv"))

In [132]:
ctaylor_finaldat = compile_dat('cc2d7ad5', 'Charlie-Taylor', 'Charlie_Taylor', checkgames = True)
if isinstance(ctaylor_finaldat, pd.DataFrame):
    ctaylor_finaldat.to_csv(os.path.join(nest_folder_def, "ctaylor_finaldat.csv"))

In [133]:
bellakotchap_finaldat = compile_dat('a39d11bc', 'Armel-Bella-Kotchap', 'Armel', checkgames = True)
if isinstance(bellakotchap_finaldat, pd.DataFrame):
    bellakotchap_finaldat.to_csv(os.path.join(nest_folder_def, "bellakotchap_finaldat.csv"))

## Spurs

In [274]:
solanke_finaldat = compile_dat('e77dc3b2', 'Dominic-Solanke', 'Dominic_Solanke', checkgames = True)
if isinstance(solanke_finaldat, pd.DataFrame):
    solanke_finaldat.to_csv(os.path.join(nest_folder, "solanke_finaldat.csv"))

In [275]:
richarlison_finaldat = compile_dat('fa031b34', 'Richarlison', 'Richarlison', checkgames = True)
if isinstance(richarlison_finaldat, pd.DataFrame):
    richarlison_finaldat.to_csv(os.path.join(nest_folder, "richarlison_finaldat.csv"))

In [276]:
son_finaldat = compile_dat('92e7e919', 'Son-Heung-min', 'Heung', checkgames = True)
if isinstance(son_finaldat, pd.DataFrame):
    son_finaldat.to_csv(os.path.join(nest_folder, "son_finaldat.csv"))

In [277]:
maddison_finaldat = compile_dat('ee38d9c5', 'James-Maddison', 'James_Maddison', checkgames = True)
if isinstance(maddison_finaldat, pd.DataFrame):
    maddison_finaldat.to_csv(os.path.join(nest_folder, "maddison_finaldat.csv"))

In [278]:
bissouma_finaldat = compile_dat('6c203af0', 'Yves-Bissouma', 'Yves_Bissouma', checkgames = True)
if isinstance(bissouma_finaldat, pd.DataFrame):
    bissouma_finaldat.to_csv(os.path.join(nest_folder, "bissouma_finaldat.csv"))

In [279]:
bjohnson_finaldat = compile_dat('0cd31129', 'Brennan-Johnson', 'Brennan_Johnson', checkgames = True)
if isinstance(bjohnson_finaldat, pd.DataFrame):
    bjohnson_finaldat.to_csv(os.path.join(nest_folder, "bjohnson_finaldat.csv"))

In [280]:
kulusevski_finaldat = compile_dat('df3cda47', 'Dejan-Kulusevski', 'Dejan_Kulusevski', checkgames = True)
if isinstance(kulusevski_finaldat, pd.DataFrame):
    kulusevski_finaldat.to_csv(os.path.join(nest_folder, "kulusevski_finaldat.csv"))

In [281]:
odobert_finaldat = compile_dat('35516eac', 'Wilson-Odobert', 'Wilson_Odobert', checkgames = True)
if isinstance(odobert_finaldat, pd.DataFrame):
    odobert_finaldat.to_csv(os.path.join(nest_folder, "odobert_finaldat.csv"))

In [282]:
pmsarr_finaldat = compile_dat('feb5d972', 'Pape-Matar-Sarr', 'Pape Matar', checkgames = True)
if isinstance(pmsarr_finaldat, pd.DataFrame):
    pmsarr_finaldat.to_csv(os.path.join(nest_folder, "pmsarr_finaldat.csv"))

In [283]:
werner_finaldat = compile_dat('49fe9070', 'Timo-Werner', 'Timo_Werner', checkgames = True)
if isinstance(werner_finaldat, pd.DataFrame):
    werner_finaldat.to_csv(os.path.join(nest_folder, "werner_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [284]:
bentancur_finaldat = compile_dat('3b8674e6', 'Rodrigo=Bentacur', 'Rodrigo_Bentancur', checkgames = True)
if isinstance(bentancur_finaldat, pd.DataFrame):
    bentancur_finaldat.to_csv(os.path.join(nest_folder, "bentancur_finaldat.csv"))

Spurs Defenders

In [134]:
romero_finaldat = compile_dat('a3d94a58', 'Cristian-Romero', 'Cristian_Romero', checkgames = True)
if isinstance(romero_finaldat, pd.DataFrame):
    romero_finaldat.to_csv(os.path.join(nest_folder_def, "romero_finaldat.csv"))

In [135]:
porro_finaldat = compile_dat('27d0a506', 'Pedro-Porro', 'Porro', checkgames = True)
if isinstance(porro_finaldat, pd.DataFrame):
    porro_finaldat.to_csv(os.path.join(nest_folder_def, "porro_finaldat.csv"))

In [136]:
vandeven_finaldat = compile_dat('8fe2a392', 'Micky-van-de-Ven', 'Micky', checkgames = True)
if isinstance(vandeven_finaldat, pd.DataFrame):
    vandeven_finaldat.to_csv(os.path.join(nest_folder_def, "vandeven_finaldat.csv"))

In [137]:
udogie_finaldat = compile_dat('7cd520e8', 'Destiny-Udogie', 'Destiny_Udogie', checkgames = True)
if isinstance(udogie_finaldat, pd.DataFrame):
    udogie_finaldat.to_csv(os.path.join(nest_folder_def, "udogie_finaldat.csv"))

In [138]:
bdavies_finaldat = compile_dat('44781702', 'Ben-Davies', 'Ben_Davies', checkgames = True)
if isinstance(bdavies_finaldat, pd.DataFrame):
    bdavies_finaldat.to_csv(os.path.join(nest_folder_def, "bdavies_finaldat.csv"))

The number of Premier League games played by Ben-Davies(171) does not match the number of rows in the dataframe (135)


In [139]:
reguilon_finaldat = compile_dat('3353737a', 'Sergio-Reguilon', 'Sergio_Regui', checkgames = True)
if isinstance(reguilon_finaldat, pd.DataFrame):
    reguilon_finaldat.to_csv(os.path.join(nest_folder_def, "reguilon_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


## West Ham

In [285]:
antonio_finaldat = compile_dat('ac05f970', 'Michail-Antonio', 'Michail_Antonio', checkgames = True)
if isinstance(antonio_finaldat, pd.DataFrame):
    antonio_finaldat.to_csv(os.path.join(nest_folder, "antonio_finaldat.csv"))

In [286]:
ings_finaldat = compile_dat('07802f7f', 'Danny-Ings', 'Danny_Ings', checkgames = True)
if isinstance(ings_finaldat, pd.DataFrame):
    ings_finaldat.to_csv(os.path.join(nest_folder, "ings_finaldat.csv"))

In [287]:
bowen_finaldat = compile_dat('79c84d1c', 'Jarrod-Bowen', 'Jarrod_Bowen', checkgames = True)
if isinstance(bowen_finaldat, pd.DataFrame):
    bowen_finaldat.to_csv(os.path.join(nest_folder, "bowen_finaldat.csv"))

In [288]:
soucek_finaldat = compile_dat('6613c819', 'Tomas-Soucek', 'Tomas_Soucek', checkgames = False)
if isinstance(soucek_finaldat, pd.DataFrame):
    soucek_finaldat.to_csv(os.path.join(nest_folder, "soucek_finaldat.csv"))

In [289]:
paqueta_finaldat = compile_dat('9b6f7fd5', 'Lucas-Paqueta', 'Lucas_Tolentino', checkgames = True)
if isinstance(paqueta_finaldat, pd.DataFrame):
    paqueta_finaldat.to_csv(os.path.join(nest_folder, "paqueta_finaldat.csv"))

In [290]:
kudus_finaldat = compile_dat('b62878a5', 'Mohammed-Kudus', 'Mohammed_Kudus', checkgames = True)
if isinstance(kudus_finaldat, pd.DataFrame):
    kudus_finaldat.to_csv(os.path.join(nest_folder, "kudus_finaldat.csv"))

In [291]:
edsonalvarez_finaldat = compile_dat('8b3ab7ad', 'Edson-Alvarez', 'Edson', checkgames = True)
if isinstance(edsonalvarez_finaldat, pd.DataFrame):
    edsonalvarez_finaldat.to_csv(os.path.join(nest_folder, "edsonalvarez_finaldat.csv"))

In [292]:
summerville_finaldat = compile_dat('df04eb4b', 'Crysencio-Summerville', 'Crysencio_Summerville', checkgames = True)
if isinstance(summerville_finaldat, pd.DataFrame):
    summerville_finaldat.to_csv(os.path.join(nest_folder, "summerville_finaldat.csv"))

West Ham Defenders

In [146]:
kilman_finaldat = compile_dat('d0f72bf1', 'Max-Kilman', 'Max_Kilman', checkgames = True)
if isinstance(kilman_finaldat, pd.DataFrame):
    kilman_finaldat.to_csv(os.path.join(nest_folder_def, "kilman_finaldat.csv"))

In [147]:
mavropanos_finaldat = compile_dat('00963611', 'Konstantinos-Mavropanos', 'Mavropanos', checkgames = True)
if isinstance(mavropanos_finaldat, pd.DataFrame):
    mavropanos_finaldat.to_csv(os.path.join(nest_folder_def, "mavropanos_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [148]:
coufal_finaldat = compile_dat('fdf3cb77', 'Vladimir-Coufal', 'Coufal', checkgames = True)
if isinstance(coufal_finaldat, pd.DataFrame):
    coufal_finaldat.to_csv(os.path.join(nest_folder_def, "coufal_finaldat.csv"))

In [149]:
emerson_finaldat = compile_dat('e0bc6fdc', 'Emerson-Palmieri', 'Palmieri', checkgames = True)
if isinstance(emerson_finaldat, pd.DataFrame):
    emerson_finaldat.to_csv(os.path.join(nest_folder_def, "emerson_finaldat.csv"))

In [150]:
awb_finaldat = compile_dat('9e525177', 'Aaron-Wan-Bissaka', 'Bissaka', checkgames = True)
if isinstance(awb_finaldat, pd.DataFrame):
    awb_finaldat.to_csv(os.path.join(nest_folder_def, "awb_finaldat.csv"))

In [151]:
cresswell_finaldat = compile_dat('4f974391', 'Aaron-Cresswell', 'Aaron_Cresswell', checkgames = True)
if isinstance(cresswell_finaldat, pd.DataFrame):
    cresswell_finaldat.to_csv(os.path.join(nest_folder_def, "cresswell_finaldat.csv"))

## Wolves

In [293]:
cunha_finaldat = compile_dat('dc62b55d', 'Matheus-Cunha', 'Matheus_Santos Carneiro', checkgames = True)
if isinstance(cunha_finaldat, pd.DataFrame):
    cunha_finaldat.to_csv(os.path.join(nest_folder, "cunha_finaldat.csv"))

In [294]:
bellegarde_finaldat = compile_dat('10f0fdd3', 'Jean-Ricner-Bellegarde', 'Bellegarde', checkgames = True)
if isinstance(bellegarde_finaldat, pd.DataFrame):
    bellegarde_finaldat.to_csv(os.path.join(nest_folder, "bellegarde_finaldat.csv"))

In [295]:
lemina_finaldat = compile_dat('2b471f99', 'Mario-Lemina', 'Mario_Lemina', checkgames = True)
if isinstance(lemina_finaldat, pd.DataFrame):
    lemina_finaldat.to_csv(os.path.join(nest_folder, "lemina_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [296]:
heechan_finaldat = compile_dat('169fd162', 'Hwang-Hee-chan', 'Hwang', checkgames = True)
if isinstance(heechan_finaldat, pd.DataFrame):
    heechan_finaldat.to_csv(os.path.join(nest_folder, "heechan_finaldat.csv"))

In [297]:
jgomes_finaldat = compile_dat('8b57ad2c', 'Joao-Gomes', 'Victor_Gomes', checkgames = True)
if isinstance(jgomes_finaldat, pd.DataFrame):
    jgomes_finaldat.to_csv(os.path.join(nest_folder, "jgomes_finaldat.csv"))

In [298]:
doyle_finaldat = compile_dat('5526deb1', 'Tommy-Doyle', 'Tommy_Doyle', checkgames = True)
if isinstance(doyle_finaldat, pd.DataFrame):
    doyle_finaldat.to_csv(os.path.join(nest_folder, "doyle_finaldat.csv"))

In [299]:
sarabia_finaldat = compile_dat('9744ff80', 'Pablo-Sarabia', 'Pablo_Sarabia', checkgames = True)
if isinstance(sarabia_finaldat, pd.DataFrame):
    sarabia_finaldat.to_csv(os.path.join(nest_folder, "sarabia_finaldat.csv"))

In [300]:
guedes_finaldat = compile_dat('e6bc67d7', 'Goncalo-Guedes', 'Ganchinho', checkgames = True)
if isinstance(guedes_finaldat, pd.DataFrame):
    guedes_finaldat.to_csv(os.path.join(nest_folder, "guedes_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


Wolves Defenders

In [152]:
aitnouri_finaldat = compile_dat('9b398aea', 'Rayan-Ait-Nouri', 'Rayan', checkgames = True)
if isinstance(aitnouri_finaldat, pd.DataFrame):
    aitnouri_finaldat.to_csv(os.path.join(nest_folder_def, "aitnouri_finaldat.csv"))

In [153]:
dawson_finaldat = compile_dat('3e9e06cb', 'Craig-Dawson', 'Craig_Dawson', checkgames = True)
if isinstance(dawson_finaldat, pd.DataFrame):
    dawson_finaldat.to_csv(os.path.join(nest_folder_def, "dawson_finaldat.csv"))

In [154]:
toti_finaldat = compile_dat('f0caab96', 'Toti-Gomes', 'Toti', checkgames = True)
if isinstance(toti_finaldat, pd.DataFrame):
    toti_finaldat.to_csv(os.path.join(nest_folder_def, "toti_finaldat.csv"))

In [155]:
semedo_finaldat = compile_dat('d04b94db', 'Nelson-Semedo', 'Semedo', checkgames = True)
if isinstance(semedo_finaldat, pd.DataFrame):
    semedo_finaldat.to_csv(os.path.join(nest_folder_def, "semedo_finaldat.csv"))

In [156]:
mdoherty_finaldat = compile_dat('d557d734', 'Matt-Doherty', 'Matt_Doherty', checkgames = True)
if isinstance(mdoherty_finaldat, pd.DataFrame):
    mdoherty_finaldat.to_csv(os.path.join(nest_folder_def, "mdoherty_finaldat.csv"))